In [1]:
import sqlite3
import os
import re
from math import inf
import numpy as np
from scipy import stats

folder_path = r'D:/Athletic Screen 2.0/Output Files/'
db_path     = r'D:/Athletic Screen 2.0/Output Files/movement_database_v2.db'

# fresh db
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Deleted existing database at {db_path}")

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
conn.execute("PRAGMA journal_mode=WAL;")
conn.execute("PRAGMA busy_timeout=5000;") 


# --- schemas (unchanged from your message) ---
table_schemas = {
    'CMJ': '''CREATE TABLE IF NOT EXISTS CMJ (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT,
                trial_name TEXT,
                JH_IN REAL,
                Peak_Power REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL
              )''',
    'PPU': '''CREATE TABLE IF NOT EXISTS PPU (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT,
                trial_name TEXT,
                JH_IN REAL,
                Peak_Power REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL
            )''',
    'DJ':  '''CREATE TABLE IF NOT EXISTS DJ (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT,
                trial_name TEXT,
                JH_IN REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL,
                CT REAL,
                RSI REAL
              )''',
    'SLV': '''CREATE TABLE IF NOT EXISTS SLV (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT, 
                trial_name TEXT,
                side TEXT,
                JH_IN REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL
              )''',
    'NMT': '''CREATE TABLE IF NOT EXISTS NMT (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT, 
                trial_name TEXT,
                NUM_TAPS_10s REAL,
                NUM_TAPS_20s REAL,
                NUM_TAPS_30s REAL,
                NUM_TAPS REAL
              )'''
}

for schema in table_schemas.values():
    cursor.execute(schema)

# ---------- helpers ----------
def extract_name(line):
    m = re.search(r'Data\\(.*?)[_\\]', line)
    return m.group(1) if m else None

def extract_date(line):
    m = re.search(r'\\(\d{4}-\d{2}-\d{2})_', line)
    return m.group(1) if m else None

def read_first_numeric_row_values(fobj):
    """Return list of floats from the first numeric line encountered."""
    for line in fobj:
        line = line.strip()
        if not line:
            continue
        if re.match(r'^[-+]?\d', line):   # numeric line
            return [float(tok) for tok in line.split()]
    return []

def peak_power_from_pow_file(trial_name_base):
    """
    Look for '{trial_name_base}_Power.txt' in folder_path, parse PowZ time series,
    and return its max (or None if not found/empty).
    """
    power_file = os.path.join(folder_path, f"{trial_name_base}_Power.txt")
    if not os.path.exists(power_file):
        return None

    peak = -inf
    try:
        with open(power_file, 'r') as pf:
            # Skip header lines until numeric rows start (format: ITEM <tab> X)
            for line in pf:
                line = line.strip()
                if not line:
                    continue
                if re.match(r'^\d+\s+', line):  # rows like "1\t0.00000"
                    # from now on, all lines *should* be numeric rows
                    # take the second column as PowZ value
                    parts = line.split()
                    if len(parts) >= 2:
                        val = float(parts[1])
                        if val > peak:
                            peak = val
                    # continue through rest of file
                    break
            # continue consuming rest of numeric rows
            for line in pf:
                line = line.strip()
                if not line or not re.match(r'^\d+\s+', line):
                    continue
                parts = line.split()
                if len(parts) >= 2:
                    val = float(parts[1])
                    if val > peak:
                        peak = val
    except Exception:
        return None

    return None if peak == -inf else peak

def insert_row(table, cols, vals):
    placeholders = ",".join(["?"] * len(vals))
    cursor.execute(f"INSERT INTO {table} ({','.join(cols)}) VALUES ({placeholders})", vals)

# ---------- main loop ----------
for file_name in os.listdir(folder_path):
    if not file_name.endswith('.txt'):
        continue
    if file_name.endswith('_Power.txt'):
        # handled separately when computing Peak_Power; skip in main loop
        continue

    trial_name = os.path.splitext(file_name)[0]
    # classify
    if 'CMJ' in trial_name:
        table = 'CMJ'
    elif 'PPU' in trial_name:
        table = 'PPU'
    elif 'DJ' in trial_name:
        table = 'DJ'
    elif 'SLVL' in trial_name or 'SLVR' in trial_name:
        table = 'SLV'
    elif 'NMT' in trial_name:
        table = 'NMT'
    else:
        continue

    file_path = os.path.join(folder_path, file_name)
    try:
        with open(file_path, 'r') as f:
            first_line = f.readline().strip()
            name = extract_name(first_line)
            date = extract_date(first_line)
            if not name:
                print(f"Name extraction failed for {file_name}, skipping.")
                continue

            variables = read_first_numeric_row_values(f)
            if not variables:
                print(f"No numeric data found in {file_name}, skipping.")
                continue

            print(f"Processing file: {file_name}, Variables: {variables}")

            # drop leading dummy "1"
            v = variables[1:] if variables and variables[0] == 1.0 else variables[:]

            # movement-specific handling
            if table in ('CMJ', 'PPU'):
                # expected 5 values: JH, PP_FORCEPLATE, Force@PP, Vel@PP, W/kg
                # compute Peak_Power from corresponding *_Power.txt if available
                peak_power = peak_power_from_pow_file(trial_name)
                if len(v) == 5:
                    JH, PP_FP, F_at_PP, V_at_PP, Wkg = v
                elif len(v) == 6:
                    # sometimes Peak_Power included as v[1]; keep robust fallback
                    JH = v[0]
                    if peak_power is None:
                        peak_power = v[1]
                    PP_FP, F_at_PP, V_at_PP, Wkg = v[-4:]
                else:
                    print(f"Unexpected column count for {file_name}: {len(v)}; skipping.")
                    continue

                cols = ["name","date","trial_name","JH_IN","Peak_Power","PP_FORCEPLATE","Force_at_PP","Vel_at_PP","PP_W_per_kg"]
                vals = [name, date, trial_name, JH, peak_power, PP_FP, F_at_PP, V_at_PP, Wkg]
                insert_row(table, cols, vals)
    
            elif table == 'DJ':
                # v must be exactly 7 values in this order:
                # [JH_IN, PP_FORCEPLATE, Force_at_PP, Vel_at_PP, CT, RSI, PP_W_per_kg]
                if len(v) != 7:
                    print(f"Unexpected DJ column count for {file_name}: {len(v)}; expected 7. Skipping.")
                    continue
            
                JH, PP_FP, F_at_PP, V_at_PP, CT, RSI, Wkg = v  # do not modify values (keep negatives if present)
            
                cols = [
                    "name","date","trial_name",
                    "JH_IN","PP_FORCEPLATE","Force_at_PP","Vel_at_PP",
                    "PP_W_per_kg","CT","RSI"
                ]
                vals = [
                    name, date, trial_name,
                    JH, PP_FP, F_at_PP, V_at_PP,
                    Wkg, CT, RSI
                ]
                insert_row("DJ", cols, vals)

            elif table == 'SLV':
                side = 'Left' if 'SLVL' in trial_name else 'Right'
                # SLV sometimes includes Peak_Power as the second item (making 6 vals total).
                # Your SLV schema *does not* have Peak_Power; we'll ignore it if present.
                if len(v) == 6:
                    # v = [JH, Peak, PP_FP, F_at_PP, V_at_PP, Wkg] -> drop Peak
                    JH, PP_FP, F_at_PP, V_at_PP, Wkg = v[0], v[2], v[3], v[4], v[5]
                elif len(v) == 5:
                    JH, PP_FP, F_at_PP, V_at_PP, Wkg = v
                else:
                    print(f"Unexpected SLV column count for {file_name}: {len(v)}; skipping.")
                    continue

                cols = ["name","date","trial_name","side","JH_IN","PP_FORCEPLATE","Force_at_PP","Vel_at_PP","PP_W_per_kg"]
                vals = [name, date, trial_name, side, JH, PP_FP, F_at_PP, V_at_PP, Wkg]
                insert_row(table, cols, vals)

            elif table == 'NMT':
                if len(v) != 4:
                    print(f"Unexpected NMT column count for {file_name}: {len(v)}; skipping.")
                    continue
                cols = ["name","date","trial_name","NUM_TAPS_10s","NUM_TAPS_20s","NUM_TAPS_30s","NUM_TAPS"]
                vals = [name, date, trial_name] + v
                insert_row(table, cols, vals)

    except Exception as e:
        print(f"Unexpected error with file {file_name}: {e}")

def load_power_txt(txt_path: str):
    """Read <trial>_Power.txt -> return 1-D numpy/pandas-like series of PowZ."""
    vals = []
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        # find first numeric row like: "1\t0.00000"
        in_numeric = False
        for line in f:
            line = line.strip()
            if not line:
                continue
            if not in_numeric and re.match(r'^\d+\s+', line):
                in_numeric = True
            if in_numeric and re.match(r'^\d+\s+', line):
                parts = re.split(r'\s+', line)
                if len(parts) >= 2:
                    try:
                        vals.append(float(parts[1]))  # second column = power
                    except ValueError:
                        pass
    if not vals:
        raise ValueError(f"No numeric power values in {txt_path}")
    # return a simple list or numpy array is fine for our analysis funcs
    return np.asarray(vals, dtype=float)

def analyze_power_curve(power, fs_hz: float = 1000.0):
    """Base metrics used by analyze_power_curve_advanced."""
    p = np.asarray(power, dtype=float)
    n = p.size
    t = np.arange(n) / fs_hz

    pk_idx = int(np.nanargmax(p))
    pk_val = float(p[pk_idx])

    thr10 = 0.10 * pk_val
    thr50 = 0.50 * pk_val
    thr90 = 0.90 * pk_val

    try:
        onset_idx = int(np.argmax(p >= thr10))
    except ValueError:
        onset_idx = 0
    post = p[pk_idx:]
    off_rel = np.argmax(post < thr10) if np.any(post < thr10) else (post.size - 1)
    offset_idx = pk_idx + int(off_rel)

    rising = p[:pk_idx+1]
    try:
        i10 = int(np.argmax(rising >= thr10))
        i90 = int(np.argmax(rising >= thr90))
        rise_time = (i90 - i10) / fs_hz if i90 > i10 else np.nan
        rise_slope = (0.8 * pk_val) / rise_time if rise_time and rise_time > 0 else np.nan
    except ValueError:
        i10 = i90 = None
        rise_time = np.nan
        rise_slope = np.nan

    try:
        left_idx  = int(np.argmax(rising >= thr50))
    except ValueError:
        left_idx = pk_idx
    falling = p[pk_idx:]
    try:
        right_rel = int(np.argmax(falling <= thr50))
        right_idx = pk_idx + right_rel
    except ValueError:
        right_idx = pk_idx
    fwhm_sec = (right_idx - left_idx) / fs_hz if right_idx > left_idx else np.nan

    a = max(0, onset_idx)
    b = min(n - 1, max(offset_idx, pk_idx))
    auc_joules = float(np.trapezoid(np.nan_to_num(p[a:b+1], nan=0.0), dx=1.0/fs_hz))

    weights = np.clip(p[a:b+1], a_min=0, a_max=None)
    if np.sum(weights) > 0:
        t_window = t[a:b+1]
        t_com = float(np.sum(t_window * weights) / np.sum(weights))
        t_com_norm = (t_com - t[a]) / max(1e-9, (t[b] - t[a]))
    else:
        t_com = np.nan
        t_com_norm = np.nan

    w = int(0.05 * fs_hz)
    lo = max(0, pk_idx - w)
    hi = min(n, pk_idx + w + 1)
    local = p[lo:hi]
    cv_local = float(np.std(local) / np.mean(local)) if np.mean(local) > 0 else np.nan

    return {
        "n_samples": n,
        "fs_hz": fs_hz,
        "peak_power_w": pk_val,
        "time_to_peak_s": float(t[pk_idx]),
        "rise_time_10_90_s": float(rise_time),
        "rise_slope_w_per_s": float(rise_slope),
        "fwhm_s": float(fwhm_sec),
        "auc_j": auc_joules,
        "onset_idx": a,
        "offset_idx": b,
        "peak_idx": pk_idx,
        "t_com_s": t_com,
        "t_com_norm_0to1": t_com_norm,
        "cv_local_peak": cv_local,
        "i10_idx": int(i10) if isinstance(i10, int) else None,
        "i90_idx": int(i90) if isinstance(i90, int) else None,
        "left50_idx": left_idx,
        "right50_idx": right_idx,
    }

def analyze_power_curve_advanced(power, fs_hz: float = 1000.0):
    """Adds RPD, early/late work, decay, shape stats, spectral centroid."""
    base = analyze_power_curve(power, fs_hz)
    p = np.asarray(power, dtype=float)
    n = p.size

    dp = np.gradient(p, 1.0/fs_hz)
    base["rpd_max_w_per_s"]   = float(np.nanmax(dp))
    base["time_to_rpd_max_s"] = float(np.nanargmax(dp) / fs_hz)

    a, b, pk = base["onset_idx"], base["offset_idx"], base["peak_idx"]
    auc_pre  = float(np.trapezoid(np.nan_to_num(p[a:pk+1],  nan=0.0), dx=1.0/fs_hz)) if pk >= a else np.nan
    auc_post = float(np.trapezoid(np.nan_to_num(p[pk:b+1], nan=0.0), dx=1.0/fs_hz)) if b >= pk else np.nan
    total = (0 if not np.isfinite(auc_pre) else auc_pre) + (0 if not np.isfinite(auc_post) else auc_post)
    base["auc_pre_j"] = auc_pre
    base["auc_post_j"] = auc_post
    base["work_early_pct"] = float(100.0 * auc_pre / total) if total > 0 else np.nan

    fall = p[pk:]
    thr90 = 0.90 * p[pk]
    thr10 = 0.10 * p[pk]
    i90 = int(np.argmax(fall <= thr90)) if np.any(fall <= thr90) else 0
    i10 = int(np.argmax(fall <= thr10)) if np.any(fall <= thr10) else len(fall)-1
    base["decay_90_10_s"] = (i10 - i90) / fs_hz if i10 > i90 else np.nan

    finite = np.isfinite(p)
    base["skewness"] = float(stats.skew(p[finite])) if np.any(finite) else np.nan
    base["kurtosis"] = float(stats.kurtosis(p[finite], fisher=True)) if np.any(finite) else np.nan

    x = p - np.nanmean(p)
    X = np.abs(np.fft.rfft(np.nan_to_num(x)))
    freqs = np.fft.rfftfreq(x.size, d=1.0/fs_hz)
    base["spectral_centroid_hz"] = float(np.sum(freqs * X) / max(1e-12, np.sum(X)))

    return base

metric_cols = [
    ("peak_power_w",          "REAL"),
    ("time_to_peak_s",        "REAL"),
    ("rpd_max_w_per_s",       "REAL"),
    ("time_to_rpd_max_s",     "REAL"),
    ("rise_time_10_90_s",     "REAL"),
    ("fwhm_s",                "REAL"),
    ("auc_j",                 "REAL"),
    ("work_early_pct",        "REAL"),
    ("decay_90_10_s",         "REAL"),
    ("t_com_norm_0to1",       "REAL"),
    ("skewness",              "REAL"),
    ("kurtosis",              "REAL"),
    ("spectral_centroid_hz",  "REAL"),
]

tables = ["CMJ", "DJ", "PPU", "SLV"]   # add/remove as needed

def column_exists(cur, table, col):
    cur.execute(f"PRAGMA table_info({table})")
    return any(row[1].lower() == col.lower() for row in cur.fetchall())

def ensure_metric_columns(conn, table):
    cur = conn.cursor()
    for col, sqltype in metric_cols:
        if not column_exists(cur, table, col):
            cur.execute(f"ALTER TABLE {table} ADD COLUMN {col} {sqltype}")
    conn.commit()

def update_table_with_power(conn, table, fs_hz=1000.0):
    """
    For each row in `table`, find '<trial_name>_Power.txt' and compute metrics.
    For SLV we also match on 'side' in the WHERE clause.
    """
    cur = conn.cursor()
    ensure_metric_columns(conn, table)

    # Pull identifying fields per table
    if table == "SLV":
        cur.execute("SELECT id, trial_name, side FROM SLV")
        rows = cur.fetchall()
    else:
        cur.execute(f"SELECT id, trial_name FROM {table}")
        rows = [(r[0], r[1], None) for r in cur.fetchall()]  # pad with None for uniformity

    # Prepare dynamic UPDATE
    set_clause = ", ".join([f"{c} = ?" for (c, _) in metric_cols])
    sql = f"UPDATE {table} SET {set_clause} WHERE id = ?"

    updated = 0
    missing = 0
    failed  = 0

    for rec in rows:
        id_, trial_name, _side = rec
        if not trial_name:
            missing += 1
            continue

        power_path = os.path.join(folder_path, f"{trial_name}_Power.txt")
        if not os.path.exists(power_path):
            missing += 1
            continue

        try:
            s = load_power_txt(power_path)
            m = analyze_power_curve_advanced(s, fs_hz=fs_hz)

            # Order must match metric_cols
            values = [
                m.get("peak_power_w"),
                m.get("time_to_peak_s"),
                m.get("rpd_max_w_per_s"),
                m.get("time_to_rpd_max_s"),
                m.get("rise_time_10_90_s"),
                m.get("fwhm_s"),
                m.get("auc_j"),
                m.get("work_early_pct"),
                m.get("decay_90_10_s"),
                m.get("t_com_norm_0to1"),
                m.get("skewness"),
                m.get("kurtosis"),
                m.get("spectral_centroid_hz"),
                id_,
            ]
            cur.execute(sql, values)
            updated += 1

        except Exception as e:
            print(f"⚠️  {table} id={id_} trial={trial_name}: power parse/analysis failed: {e}")
            failed += 1

    conn.commit()
    print(f"{table}: updated={updated}, missing_power_files={missing}, failed={failed}")

# ---- run the updater ----
for t in tables:
    update_table_with_power(conn, t, fs_hz=1000.0)

conn.commit()
conn.close()
print("Data successfully inserted into the database.")


Deleted existing database at D:/Athletic Screen 2.0/Output Files/movement_database_v2.db
Processing file: CMJ1.txt, Variables: [1.0, 20.0, 1880.5, 1816.18, 1035.43, 24.1]
Processing file: CMJ2.txt, Variables: [1.0, 19.2, 2338.2, 1699.74, 1375.65, 29.97]
Processing file: CMJ3.txt, Variables: [1.0, 20.0, 1880.5, 1816.18, 1035.43, 24.1]
Processing file: DJ1.txt, Variables: [1.0, 17.8, 1170.4, 1505.16, 777.62, 0.54, 1.68, 15.0]
Processing file: DJ2.txt, Variables: [1.0, 20.4, 1204.6, 1540.72, 781.86, 0.52, 1.98, 15.44]
Processing file: DJ3.txt, Variables: [1.0, 20.2, 1367.6, 2284.35, 598.69, 0.31, 3.27, 17.53]
Processing file: SLVL1.txt, Variables: [1.0, 9.8, 6168.0, 1837.5, 1486.5, 1236.1, 23.55]
Processing file: SLVL2.txt, Variables: [1.0, 10.1, 6214.0, 1772.5, 1348.9, 1314.1, 22.72]
Processing file: SLVR1.txt, Variables: [1.0, 12.2, 6554.0, 3239.2, 1751.5, 1849.4, 41.52]
Processing file: SLVR2.txt, Variables: [1.0, 11.9, 6503.0, 1689.7, 1603.9, 1053.5, 21.66]
Processing file: PPU1.txt, 

In [2]:
# Creates full report for age group comparison

import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import date
import tempfile
import docx2txt
from PIL import Image, ImageDraw, ImageFont
import os, re
from scipy import integrate, stats
import glob as globmod

# -------- style to match your dark report --------
plt.rcParams.update({
    "figure.facecolor": "#181818",
    "axes.facecolor"  : "#303030",
    "axes.edgecolor"  : "white",
    "axes.labelcolor" : "slategrey",
    "xtick.color"     : "lightgrey",
    "ytick.color"     : "lightgrey",
    "grid.color"      : "dimgrey",
    "text.color"      : "white",
})
# Corrected file paths with raw strings to handle backslashes properly
client_db_path = r'D:\Athletic Screen 2.0\Output Files\movement_database_v2.db'
reference_db_path = r'D:\Athletic Screen 2.0\Output Files\Athletic_Screen_College_data_v2.db'

# Ensure the paths are valid and accessible
if not os.path.exists(client_db_path):
    print(f"Client database not found at {client_db_path}")
if not os.path.exists(reference_db_path):
    print(f"Reference database not found at {reference_db_path}")

# Connect to the client and reference databases
client_conn = sqlite3.connect(client_db_path)
reference_conn = sqlite3.connect(reference_db_path)
client_cursor = client_conn.cursor()
reference_cursor = reference_conn.cursor()

print("Databases opened successfully.")

# Fetch the client's name from the database (assuming the 'name' column is in all tables)
client_cursor.execute("SELECT DISTINCT name FROM CMJ")  # Change table if necessary
client_name = client_cursor.fetchone()[0]  # Get the first row and first column
print(f"Client Name: {client_name}")

# ---------- build unique export paths (date-stamped, no overwrite) -----
client_cursor.execute("SELECT MAX(date) FROM CMJ WHERE name = ?", (client_name,))
assessment_date = client_cursor.fetchone()[0]          # e.g. '2025-05-22'
if not assessment_date:                                # fallback to today
    assessment_date = date.today().strftime("%Y-%m-%d")

reports_dir = r'G:\My Drive\Athletic Screen 2.0 Reports\College Reports'
os.makedirs(reports_dir, exist_ok=True)

parts = client_name.split(', ')
client_name_rev = f"{parts[1]}_{parts[0]}" if len(parts) == 2 else client_name
base_name = f"Athletic_Report_{client_name_rev}_{assessment_date}"

output_filename = os.path.join(reports_dir, base_name + ".docx")
img_output_directory = os.path.join(reports_dir, "Images", base_name)
os.makedirs(img_output_directory, exist_ok=True)

# auto-increment if the same file already exists
counter = 1
while os.path.exists(output_filename):
    output_filename = os.path.join(
        reports_dir, f"{base_name}_{counter}.docx"
    )
    img_output_directory = os.path.join(
        reports_dir, "Images", f"{base_name}_{counter}"
    )
    os.makedirs(img_output_directory, exist_ok=True)
    counter += 1
# ----------------------------------------------------------------------

# Helper function to calculate percentile
def calculate_percentile(value, reference_data):
    return stats.percentileofscore(reference_data, value)

# ─── UPDATED generate_bar_graph ────────────────────────────────────────────────
def generate_bar_graph(variable, client_value, reference_data, title, tmpdirname):
    """
    Blue bars  = reference distribution
    ─ red      = client MAX (best trial **within the same movement table**)
    ─ violet   = client MEAN (average of those trials)

    • No code outside this function needs to change.
    • If the caller still passes one score, that is fine; this function looks
      up any matching trials on the same assessment day and combines them.
    • RSI histograms use 0.25-wide bins so you see bars at 1.25, 1.50, 1.75 …
    """
    import numpy as np, matplotlib.pyplot as plt, os, sqlite3, re

    # ───────────── lookup: which movement table are we dealing with? ─────────
    column_table_map = {"CMJ": ["CMJ"], "DJ": ["DJ"], "PPU": ["PPU"], "SLV": ["SLV"], "NMT": ["NMT"]}
    table_guess = None
    for tbl in column_table_map:          # CMJ, DJ, SLV, NMT
        try:
            reference_cursor.execute(f"SELECT COUNT(*) FROM {tbl}")
            if reference_cursor.fetchone()[0] == len(reference_data):
                table_guess = tbl
                break
        except sqlite3.OperationalError:
            continue
    if table_guess is None:               # fallback
        for tbl in column_table_map:
            try:
                client_cursor.execute(f"SELECT 1 FROM {tbl} LIMIT 1")
                table_guess = tbl
                break
            except sqlite3.OperationalError:
                continue

    # ───────────── gather all trials for this athlete / table / day ──────────
    scores = []
    if table_guess:
        # Some tables (e.g., NMT) may not have trial_name. If so, fall back safely.
        try:
            client_cursor.execute(
                f"SELECT trial_name FROM {table_guess} WHERE name=? LIMIT 1",
                (client_name,),
            )
            row = client_cursor.fetchone()
            date_prefix = None
            if row and row[0]:
                m = re.match(r"(\d{4}[-_]\d{2}[-_]\d{2})", row[0])
                date_prefix = m.group(1) if m else None
    
            if date_prefix:
                q = f"SELECT {variable} FROM {table_guess} WHERE name=? AND trial_name LIKE ?"
                client_cursor.execute(q, (client_name, f"{date_prefix}%"))
            else:
                q = f"SELECT {variable} FROM {table_guess} WHERE name=?"
                client_cursor.execute(q, (client_name,))
    
            scores = [r[0] for r in client_cursor.fetchall() if r[0] is not None]
        except sqlite3.OperationalError:
            # No trial_name column (likely NMT) → just use the single value
            scores = [client_value]
    
    # fallback if still empty
    if not scores:
        scores = [client_value]


    scores = np.asarray(scores, dtype=float)
    c_mean = scores.mean()
    
    # --- choose the extreme we draw as the red line ---------------------------
    if variable.upper() == "CT":        # Contact-Time → use the LOWEST value
        c_extreme = scores.min()
        extreme_label = "Client Min"
    else:                               # every other metric → highest value
        c_extreme = scores.max()
        extreme_label = "Client Max"

    perc_mean = calculate_percentile(c_mean, reference_data)

    # ──────────────────────────── plotting begins ────────────────────────────
    plt.figure(facecolor="#181818")
    ax = plt.subplot(111, facecolor="#303030")

    reference_plotted = False  # flag to avoid double-plotting

    # ---------- RSI special case: 0.25-wide bars & custom ticks -------------
    if variable.upper() == "RSI":
        lo = np.floor(reference_data.min() / 0.25) * 0.25
        hi = np.ceil(reference_data.max() / 0.25) * 0.25
        bins = np.arange(lo, hi + 0.25, 0.25)      # bin edges
        centers = bins[:-1]                         # bar positions

        counts, _ = np.histogram(reference_data, bins=bins)
        ax.bar(
            centers,
            counts,
            width=0.25,
            align="edge",
            color="cornflowerblue",
            alpha=0.7,
            edgecolor="white",
            label="Reference",
        )
        # after counts, _ = np.histogram(...)
        for x, h in zip(centers, counts):
            if h == 0:                        # empty bin → draw thin outline
                ax.bar(x, 1e-6, width=.25, align='edge',
                       color='none', edgecolor='#404040', linewidth=.5)

        ax.set_xticks(centers)
        ax.set_xticklabels([f"{x:.2f}" for x in centers], color="lightgrey")

        reference_plotted = True  # we already drew the reference bars
    else:
        bins = 20  # default bin count

    # ---------- draw reference histogram when not plotted above -------------
    if not reference_plotted:
        ax.hist(
            reference_data,
            bins=bins,
            color="cornflowerblue",
            alpha=0.7,
            edgecolor="white",
            label="Reference",
        )

    # ---------- client mean / max lines -------------------------------------
    ax.axvline(c_extreme, color="red", ls="--", lw=2, label="Client Max")
    ax.axvline(c_mean, color="violet", ls="--", lw=2, label="Client Mean")

    # ---------- cosmetics ----------------------------------------------------
    ax.set_xlabel(variable.replace("_", " "), color="slategrey")
    ax.set_ylabel("Frequency", color="slategrey")
    ax.tick_params(axis="x", colors="lightgrey")
    ax.tick_params(axis="y", colors="lightgrey")
    ax.grid(color="dimgrey")

    txt = (
        f"Percentile (mean): {perc_mean:.1f}%"
        f"\nMean: {c_mean:.2f}"
        f"\nMax:  {c_extreme:.2f}"
    )
    plt.text(
        0.95,
        0.05,
        txt,
        ha="right",
        va="bottom",
        transform=ax.transAxes,
        color="white",
        fontsize=9,
        backgroundcolor="#181818",
    )

    ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")

    # ---------- save ---------------------------------------------------------
    out_path = os.path.join(tmpdirname, f"{variable}_histogram.png")
    plt.savefig(out_path, bbox_inches="tight", facecolor="#181818")
    plt.close()
    return out_path

# Function to generate scatter plot for CMJ
def generate_scatter_plot(client_data, reference_data, x_var, y_var, title, tmpdirname):
    plt.figure(facecolor='#181818', figsize=(6, 6))
    ax = plt.subplot(111, facecolor='#303030')

    # Create scatter plot for reference data (cornflower blue)
    ax.scatter(reference_data[x_var], reference_data[y_var], label='Reference', alpha=0.5, color='cornflowerblue')

    # Create scatter plot for client data (red)
    ax.scatter(client_data[x_var], client_data[y_var], label='Client', color='red', edgecolors='black', s=100)

    # Set axis labels, replacing underscores with spaces
    ax.set_xlabel(x_var.replace('_', ' '), color='slategrey')
    ax.set_ylabel(y_var.replace('_', ' '), color='slategrey')

    # Dynamically set ticks and numbers to light grey
    ax.tick_params(axis='x', colors='lightgrey')
    ax.tick_params(axis='y', colors='lightgrey')

    # Add vertical and horizontal reference lines (light grey)
    ax.axvline(x=np.mean(reference_data[x_var]), color='lightgrey', linestyle='--', linewidth=1)
    ax.axhline(y=np.mean(reference_data[y_var]), color='lightgrey', linestyle='--', linewidth=1)

    # Customize grid style
    ax.grid(color='dimgrey')

    # Add legend
    ax.legend(facecolor='black', edgecolor='grey', prop={'size': 'small'}, labelcolor='grey')

    # Save scatter plot to file
    scatter_filename = os.path.join(tmpdirname, 'cmj_scatter.png')
    plt.savefig(scatter_filename, bbox_inches='tight', facecolor='#181818')
    plt.close()

    return scatter_filename


def load_power_txt(txt_path: str) -> pd.Series:
    """
    Parse exported power file (like your example). Returns a pandas Series of power.
    Assumes: header lines, then a line starting with 'ITEM', then data rows:
             <index>\t<value>
    Skips rows without a numeric second field.
    """
    power_vals = []
    in_data = False
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            line = raw.strip()
            if not in_data:
                if line.startswith("ITEM"):
                    in_data = True
                continue
            # from here on, try to parse the last field as a float
            if not line:
                continue
            parts = re.split(r"\t+", line)
            if len(parts) < 2:
                # sometimes a blank power cell exists on the first data row — skip it
                continue
            try:
                val = float(parts[-1])
                power_vals.append(val)
            except ValueError:
                # non-numeric tail → ignore
                continue
    if not power_vals:
        raise ValueError(f"No power values parsed from {txt_path}")
    return pd.Series(power_vals, name="Power")

def analyze_power_curve(power: pd.Series, fs_hz: float = 1000.0) -> dict:
    """
    Compute useful shape/temporal features.
    power: Series of power (W). fs_hz: sampling rate (Hz). If unknown, 1000 Hz is typical.
    Returns a dict of metrics.
    """
    p = np.asarray(power, dtype=float)
    n = p.size
    t = np.arange(n) / fs_hz

    # basic
    p_peak_idx = int(np.nanargmax(p))
    p_peak     = float(p[p_peak_idx])
    t_peak     = float(t[p_peak_idx])

    # onset/offset via % of peak (robust to baseline drift)
    thr10 = 0.10 * p_peak
    thr50 = 0.50 * p_peak
    thr90 = 0.90 * p_peak

    # first index above 10% of peak
    try:
        onset_idx = int(np.argmax(p >= thr10))
    except ValueError:
        onset_idx = 0
    # first index after peak that falls below 10% (or end)
    post = p[p_peak_idx:]
    off_rel = np.argmax(post < thr10) if np.any(post < thr10) else (post.size - 1)
    offset_idx = p_peak_idx + int(off_rel)

    # 10–90% rise time on rising limb
    rising = p[:p_peak_idx+1]
    try:
        i10 = int(np.argmax(rising >= thr10))
        i90 = int(np.argmax(rising >= thr90))
        rise_time = (i90 - i10) / fs_hz if i90 > i10 else np.nan
        rise_slope = (0.8 * p_peak) / rise_time if rise_time and rise_time > 0 else np.nan
    except ValueError:
        i10 = i90 = None
        rise_time = np.nan
        rise_slope = np.nan

    # FWHM (50% of peak) width
    # left crossing
    try:
        left_idx  = int(np.argmax(rising >= thr50))
    except ValueError:
        left_idx = p_peak_idx
    # right crossing
    falling = p[p_peak_idx:]
    try:
        right_rel = int(np.argmax(falling <= thr50))
        right_idx = p_peak_idx + right_rel
    except ValueError:
        right_idx = p_peak_idx
    fwhm_sec = (right_idx - left_idx) / fs_hz if right_idx > left_idx else np.nan

    # Work/impulse of power (area under curve) over the active window
    a = max(0, onset_idx)
    b = min(n - 1, max(offset_idx, p_peak_idx))
    auc_joules = float(np.trapezoid(np.nan_to_num(p[a:b+1], nan=0.0), dx=1.0/fs_hz))

    # timing “balance”: center of mass of power curve (0..1)
    # (earlier vs. later power concentration)
    weights = p[a:b+1].clip(min=0)
    if weights.sum() > 0:
        t_window = t[a:b+1]
        t_com = float(np.sum(t_window * weights) / np.sum(weights))
        t_com_norm = (t_com - t[a]) / max(1e-9, (t[b] - t[a]))
    else:
        t_com = np.nan
        t_com_norm = np.nan

    # variability around peak (local coefficient of variation in ±50 ms)
    w = int(0.05 * fs_hz)
    lo = max(0, p_peak_idx - w)
    hi = min(n, p_peak_idx + w + 1)
    local = p[lo:hi]
    cv_local = float(np.std(local) / np.mean(local)) if np.mean(local) > 0 else np.nan

    return {
        "n_samples": n,
        "fs_hz": fs_hz,
        "peak_power_w": p_peak,
        "time_to_peak_s": t_peak,
        "rise_time_10_90_s": float(rise_time),
        "rise_slope_w_per_s": float(rise_slope),
        "fwhm_s": float(fwhm_sec),
        "auc_j": auc_joules,
        "onset_idx": a,
        "offset_idx": b,
        "peak_idx": p_peak_idx,
        "t_com_s": t_com,
        "t_com_norm_0to1": t_com_norm,
        "cv_local_peak": cv_local,
        "i10_idx": int(i10) if isinstance(i10, int) else None,
        "i90_idx": int(i90) if isinstance(i90, int) else None,
        "left50_idx": left_idx,
        "right50_idx": right_idx,
    }

def plot_power_curve(power: pd.Series,
                     metrics: dict,
                     out_path: str,
                     title: str = "Power Curve",
                     annotate: bool = True):
    """
    Plot power vs. time with annotations (peak, 10–90 rise, FWHM, AUC window).
    """
    p = np.asarray(power, dtype=float)
    t = np.arange(p.size) / metrics["fs_hz"]

    fig, ax = plt.subplots(figsize=(6, 3.6))
    ax.plot(t, p, lw=2, label="Power")
    ax.grid(True)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Power (W)")
    ax.set_title(title, pad=8, color="white")

    # annotate regions
    a, b = metrics["onset_idx"], metrics["offset_idx"]
    ax.axvspan(t[a], t[b], color="white", alpha=0.07, label="active window")

    # peak
    pk = metrics["peak_idx"]
    ax.plot([t[pk]], [p[pk]], "o", ms=6, color="tomato", label="Peak")
    ax.axhline(p[pk]*0.5, ls="--", lw=1, color="grey")
    ax.vlines([t[metrics["left50_idx"]], t[metrics["right50_idx"]]],
              ymin=0, ymax=p[pk]*0.5, linestyles="--", colors="grey", lw=1)

    # 10–90 rise (if available)
    if metrics["i10_idx"] is not None and metrics["i90_idx"] is not None:
        ax.plot([t[metrics["i10_idx"]], t[metrics["i90_idx"]]],
                [p[metrics["i10_idx"]], p[metrics["i90_idx"]]],
                lw=3, color="deepskyblue", label="10–90% rise")

    if annotate:
        txt = (f"Peak: {metrics['peak_power_w']:.1f} W @ {metrics['time_to_peak_s']:.3f} s"
               f"\nRise 10–90: {metrics['rise_time_10_90_s']:.3f} s"
               f"\nFWHM: {metrics['fwhm_s']:.3f} s"
               f"\nWork (AUC): {metrics['auc_j']:.1f} J"
               f"\nTiming COM: {metrics['t_com_norm_0to1']:.2f} (0 early…1 late)")
        ax.text(0.99, 0.02, txt, ha="right", va="bottom",
                transform=ax.transAxes, fontsize=9, color="white",
                bbox=dict(boxstyle="round,pad=0.25", facecolor="#181818", edgecolor="#444"))

    ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

def overlay_power_trials(traces: list[pd.Series],
                         fs_hz: float,
                         out_path: str,
                         title: str = "Power (all trials)",
                         align: str = "peak",          # "peak", "onset10", or "none"
                         window_s: tuple[float, float] | None = (0.30, 0.40),
                         show_mean: bool = True):
    """
    Overlay multiple power traces and align them in time.

    align:
      - "peak"    → align each trial's max power to t=0
      - "onset10" → align first sample ≥ 10% of that trial's peak to t=0
      - "none"    → no alignment; left edges at t=0

    window_s: (pre, post) seconds to show around t=0 (None to show full extent)
    """

    # --- convert input to arrays; find alignment index per trial --------------
    arrs = [np.asarray(s, dtype=float) for s in traces if len(s) > 0]
    if not arrs:
        raise ValueError("overlay_power_trials: no non-empty traces provided")

    def _align_index(x: np.ndarray) -> int:
        if align == "none":
            return 0
        # guard against NaNs
        if not np.any(np.isfinite(x)):
            return 0
        # peak index
        try:
            pk = int(np.nanargmax(x))
        except ValueError:
            pk = 0
        if align == "peak":
            return pk
        elif align == "onset10":
            thr = 0.10 * (x[pk] if np.isfinite(x[pk]) else np.nanmax(x))
            # first index >= 10% of that trial's peak
            idx = int(np.argmax(x >= thr)) if np.any(x >= thr) else 0
            return idx
        else:
            return 0

    align_idx = [ _align_index(x) for x in arrs ]

    # --- build a padded matrix so all alignment points land at the same column
    max_left  = max(align_idx)                                  # largest left padding needed
    right_len = [len(x) - i for x, i in zip(arrs, align_idx)]   # samples from align idx to end
    max_right = max(right_len)
    L = max_left + max_right                                     # total aligned length

    aligned = np.full((len(arrs), L), np.nan)
    for r, (x, i0) in enumerate(zip(arrs, align_idx)):
        start = max_left - i0
        aligned[r, start:start+len(x)] = x

    # --- time vector: t=0 at the common alignment column ---------------------
    t = (np.arange(L) - max_left) / fs_hz

    # --- optional cropping around t=0 ----------------------------------------
    if window_s is not None:
        pre, post = window_s
        i_lo = max(0, int(np.floor((-pre)  * fs_hz)) + max_left)
        i_hi = min(L, int(np.ceil( (post) * fs_hz)) + max_left)
        aligned = aligned[:, i_lo:i_hi]
        t = t[i_lo:i_hi]
        
    # --- plot ----------------------------------------------------------------
    fig, ax = plt.subplots(figsize=(6, 3.6))
    
    # consistent y-limits (same scale across trials)
    y_max = np.nanmax(aligned)
    y_min = np.nanmin(aligned)
    
    # NEW: mask-based plotting so left/right NaNs don't truncate the curve visually
    coverage_flags = []  # we'll use this to warn about very short traces
    for i in range(aligned.shape[0]):
        row = aligned[i]
        mask = np.isfinite(row)
        cov = np.count_nonzero(mask) / max(1, len(row))
        coverage_flags.append(cov)
        if np.count_nonzero(mask) >= 2:
            ax.plot(t[mask], row[mask], lw=1.2, alpha=0.6)
    
    # optional: highlight which trials are very short (<50% of the window)
    short = [idx for idx, c in enumerate(coverage_flags, start=1) if c < 0.5]
    if short:
        print(f"⚠️ overlay_power_trials: {len(short)} trial(s) with <50% coverage in the window: {short}. "
              f"Consider changing window_s or checking the export files.")
    
    if show_mean:
        mean_curve = np.nanmean(aligned, axis=0)
        ax.plot(t, mean_curve, lw=2.2, color="cyan", label="Mean")
    
    # vertical line at alignment point
    ax.axvline(0.0, color="grey", lw=1, ls="--",
               label=("Aligned peak" if align == "peak" else "Aligned onset"))
    
    ax.grid(True)
    ax.set_xlabel("Time (s, aligned)")
    ax.set_ylabel("Power (W)")
    ax.set_title(title, pad=8, color="white")
    if show_mean:
        ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")
    
    ax.set_ylim(y_min - 0.05 * abs(y_max - y_min), y_max + 0.05 * abs(y_max - y_min))
    
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

def find_power_files(movement: str,
                     base_dir: str = r'D:\Athletic Screen 2.0\Output Files') -> list[str]:
    """
    Finds exported power files for a movement. Supports:
      CMJ_Power.txt, CMJ1_Power.txt, CMJ2_Power.txt, ...
      DJ_Power.txt,  DJ*_Power.txt
      SLV_Power.txt, SLV*_Power.txt
    Returns a sorted (unique) list of full paths.
    """
    pats = [f"{movement}_Power.txt", f"{movement}*_Power.txt"]
    files = []
    for p in pats:
        files += globmod.glob(os.path.join(base_dir, p))
    return sorted(set(files))

def build_aligned_matrix(traces: list[pd.Series],
                         fs_hz: float,
                         align: str = "peak",
                         window_s: tuple[float, float] | None = (0.30, 0.40)):
    """
    Returns (aligned 2D array [n_trials x T], time vector [T], indices_info[list]).
    Time is centered so the chosen alignment point is at t=0.
    """
    arrs = [np.asarray(s, dtype=float) for s in traces if len(s) > 0]
    if not arrs:
        raise ValueError("build_aligned_matrix: no non-empty traces")

    def _align_idx(x: np.ndarray) -> int:
        if align == "none":
            return 0
        if not np.any(np.isfinite(x)):
            return 0
        pk = int(np.nanargmax(x))
        if align == "peak":
            return pk
        elif align == "onset10":
            thr = 0.10 * (x[pk] if np.isfinite(x[pk]) else np.nanmax(x))
            return int(np.argmax(x >= thr)) if np.any(x >= thr) else 0
        return 0

    aidx = [_align_idx(x) for x in arrs]
    max_left = max(aidx)
    right_len = [len(x) - i for x, i in zip(arrs, aidx)]
    max_right = max(right_len)
    L = max_left + max_right

    aligned = np.full((len(arrs), L), np.nan)
    for r, (x, i0) in enumerate(zip(arrs, aidx)):
        start = max_left - i0
        aligned[r, start:start+len(x)] = x

    t = (np.arange(L) - max_left) / fs_hz

    if window_s is not None:
        pre, post = window_s
        i_lo = max(0, int(np.floor((-pre)  * fs_hz)) + max_left)
        i_hi = min(L, int(np.ceil( (post) * fs_hz)) + max_left)
        return aligned[:, i_lo:i_hi], t[i_lo:i_hi], aidx
    return aligned, t, aidx

def mean_aligned_curve(traces: list[pd.Series],
                       fs_hz: float,
                       align: str = "peak",
                       window_s: tuple[float, float] | None = (0.30, 0.40)) -> pd.Series:
    aligned, t, _ = build_aligned_matrix(traces, fs_hz, align, window_s)
    mean_curve = np.nanmean(aligned, axis=0)
    return pd.Series(mean_curve, index=t, name="Power")

def analyze_power_curve_advanced(power: pd.Series, fs_hz: float = 1000.0) -> dict:
    """
    Extends your analyze_power_curve with additional, practical features:
      • rpd_max (max rate of power development) & time to RPDmax
      • AUC early (pre-peak) / late (post-peak), % early work
      • decay_90_10 on falling limb
      • skewness, kurtosis
      • spectral centroid (how ‘fast’ the curve is in frequency domain)
    """
    base = analyze_power_curve(power, fs_hz)
    p = np.asarray(power, dtype=float)
    n = p.size
    t = np.arange(n) / fs_hz

    # RPD
    dp = np.gradient(p, 1.0/fs_hz)
    rpd_max = float(np.nanmax(dp))
    rpd_idx = int(np.nanargmax(dp))
    base["rpd_max_w_per_s"]   = rpd_max
    base["time_to_rpd_max_s"] = rpd_idx / fs_hz

    # Early/late work around peak (use on/peak/off from base)
    a, b, pk = base["onset_idx"], base["offset_idx"], base["peak_idx"]
    auc_pre  = float(np.trapezoid(np.nan_to_num(p[a:pk+1],  nan=0.0), dx=1.0/fs_hz)) if pk >= a else np.nan
    auc_post = float(np.trapezoid(np.nan_to_num(p[pk:b+1], nan=0.0), dx=1.0/fs_hz)) if b >= pk else np.nan
    total    = (auc_pre if np.isfinite(auc_pre) else 0) + (auc_post if np.isfinite(auc_post) else 0)
    base["auc_pre_j"]      = auc_pre
    base["auc_post_j"]     = auc_post
    base["work_early_pct"] = float(100.0 * auc_pre / total) if total > 0 else np.nan

    # Decay time 90→10% of peak on falling limb
    peak_val = p[pk]
    fall = p[pk:]
    thr90 = 0.90 * peak_val
    thr10 = 0.10 * peak_val
    i90 = int(np.argmax(fall <= thr90)) if np.any(fall <= thr90) else 0
    i10 = int(np.argmax(fall <= thr10)) if np.any(fall <= thr10) else len(fall)-1
    base["decay_90_10_s"] = (i10 - i90) / fs_hz if i10 > i90 else np.nan

    # Shape stats
    finite = np.isfinite(p)
    base["skewness"] = float(stats.skew(p[finite])) if np.any(finite) else np.nan
    base["kurtosis"] = float(stats.kurtosis(p[finite], fisher=True)) if np.any(finite) else np.nan

    # Spectral centroid
    x = p - np.nanmean(p)
    X = np.abs(np.fft.rfft(np.nan_to_num(x)))
    freqs = np.fft.rfftfreq(x.size, d=1.0/fs_hz)
    base["spectral_centroid_hz"] = float(np.sum(freqs * X) / max(1e-12, np.sum(X)))

    return base

def add_power_analysis_section(doc: Document,
                               movement: str,
                               traces: list[pd.Series],
                               fs_hz: float,
                               tmpdirname: str,
                               reference_cursor,
                               reference_table: str):
    """
    1) Overlay (peaks aligned) + mean curve
    2) Annotated mean power curve
    3) Table of per-trial metrics + Mean/SD
    4) Adds reference percentile for peak power (vs PP_FORCEPLATE in ref DB)
    """
    # --- 1) Overlay (aligned at peak) ---------------------------------------
    overlay_png = os.path.join(tmpdirname, f"{movement}_power_overlay.png")
    overlay_power_trials(
        traces, fs_hz=fs_hz, out_path=overlay_png,
        title=f"{movement} Power – All Trials (peaks aligned)",
        align="peak", window_s=(0.30, 0.40), show_mean=True
    )
    doc.add_paragraph("Power Curves (aligned at peak)", style="Heading 2")
    doc.add_picture(overlay_png, width=Inches(6))

    # --- 2) Mean curve (aligned) + annotated plot ---------------------------
    mean_series = mean_aligned_curve(traces, fs_hz, align="peak", window_s=(0.30, 0.40))
    mean_metrics = analyze_power_curve_advanced(mean_series, fs_hz=fs_hz)

    mean_png = os.path.join(tmpdirname, f"{movement}_power_mean_annotated.png")
    plot_power_curve(mean_series, mean_metrics, mean_png,
                     title=f"{movement} – Mean Power (aligned)")
    doc.add_paragraph("Mean Power Curve (annotated)", style="Heading 3")
    doc.add_picture(mean_png, width=Inches(6))

    # --- 3) Per-trial metrics & summary -------------------------------------
    per = [analyze_power_curve_advanced(s, fs_hz=fs_hz) for s in traces]
    df  = pd.DataFrame(per)

    # Pick the important rows for the document (order here = row order in table)
    metric_rows = [
        ("Peak Power (W)",            "peak_power_w",        "{:.0f}"),
        ("Time to Peak (s)",          "time_to_peak_s",      "{:.3f}"),
        ("RPD max (W/s)",             "rpd_max_w_per_s",     "{:.0f}"),
        ("Time to RPD max (s)",       "time_to_rpd_max_s",   "{:.3f}"),
        ("Rise 10–90% (s)",           "rise_time_10_90_s",   "{:.3f}"),
        ("FWHM (s)",                  "fwhm_s",              "{:.3f}"),
        ("Work (AUC, J)",             "auc_j",               "{:.0f}"),
        ("Early work (%)",            "work_early_pct",      "{:.1f}"),
        ("Decay 90→10% (s)",          "decay_90_10_s",       "{:.3f}"),
        ("Timing CoM (0…1)",          "t_com_norm_0to1",     "{:.2f}"),
        ("Skewness",                  "skewness",            "{:.2f}"),
        ("Kurtosis",                  "kurtosis",            "{:.2f}"),
        ("Spectral centroid (Hz)",    "spectral_centroid_hz","{:.2f}"),
    ]

    # table: Metric | Trial1 | Trial2 | ... | Mean | SD
    tbl = doc.add_table(rows=1 + len(metric_rows), cols=2 + len(traces))
    tbl.style = "Light List" if "Light List" in [s.name for s in doc.styles] else tbl.style
    # header
    hdr = tbl.rows[0].cells
    hdr[0].text = "Metric"
    for i in range(len(traces)):
        hdr[1+i].text = f"Trial {i+1}"
    hdr[-1].text = "Mean ± SD"

    # body
    for r, (label, key, fmt) in enumerate(metric_rows, start=1):
        row_cells = tbl.rows[r].cells
        row_cells[0].text = label
        vals = df.get(key, pd.Series([np.nan]*len(traces))).values
        for i, v in enumerate(vals):
            row_cells[1+i].text = (fmt.format(v) if np.isfinite(v) else "—")
        mu = np.nanmean(vals)
        sd = np.nanstd(vals, ddof=1) if np.count_nonzero(np.isfinite(vals)) > 1 else np.nan
        row_cells[-1].text = (f"{fmt.format(mu)} ± {fmt.format(sd)}"
                              if np.isfinite(mu) and np.isfinite(sd) else
                              (fmt.format(mu) if np.isfinite(mu) else "—"))

    # small spacer
    doc.add_paragraph("")

    # --- 4) Reference percentile for peak power (vs. DB PP_FORCEPLATE) ------
    try:
        reference_cursor.execute(f"SELECT PP_FORCEPLATE FROM {reference_table} WHERE PP_FORCEPLATE IS NOT NULL")
        ref_pp = np.array([r[0] for r in reference_cursor.fetchall()], dtype=float)
    except sqlite3.OperationalError:
        ref_pp = np.array([])

    if ref_pp.size:
        # use the highest peak among trials (or mean peak if you prefer)
        trial_peaks = df["peak_power_w"].values
        best_peak   = float(np.nanmax(trial_peaks)) if trial_peaks.size else np.nan
        pctl        = percentile_vs_reference(best_peak, ref_pp) if np.isfinite(best_peak) else np.nan
        doc.add_paragraph(f"Reference percentile (peak power): {pctl:.1f}%", style="Intense Quote")

def percentile_vs_reference(value: float, reference_values: np.ndarray) -> float:
    """
    Wrapper for percentile (1..99) using scipy-like methodology.
    """
    return stats.percentileofscore(reference_values, value)

# Modified function to generate a histogram comparing left and right leg data
def generate_slv_histogram(variable, left_value, right_value,
                           reference_data, title, tmpdirname):
    """
    Blue bars  = reference distribution
    ─ green    = client LEFT (latest trial value you passed in)
    ─ orange   = client RIGHT

    Text box shows, for each side:
        • mean across all trials from the same assessment day
        • max across those trials
        • percentile of that mean vs. the reference distribution
    """
    import numpy as np, matplotlib.pyplot as plt, os, re, sqlite3

    # ── helper – get all trials for a given side on the same assessment day ──
    def _fetch_side_vals(side):
        # detect yyyy-mm-dd prefix in the first trial_name for this side
        client_cursor.execute(
            "SELECT trial_name FROM SLV WHERE name=? AND side=? LIMIT 1",
            (client_name, side)
        )
        row = client_cursor.fetchone()
        date_prefix = None
        if row and row[0]:
            m = re.match(r'(\d{4}[-_]\d{2}[-_]\d{2})', row[0])
            date_prefix = m.group(1) if m else None

        if date_prefix:
            q = f"SELECT {variable} FROM SLV WHERE name=? AND side=? AND trial_name LIKE ?"
            client_cursor.execute(q, (client_name, side, f'{date_prefix}%'))
        else:
            q = f"SELECT {variable} FROM SLV WHERE name=? AND side=?"
            client_cursor.execute(q, (client_name, side))

        return [r[0] for r in client_cursor.fetchall() if r[0] is not None]

    # pull all trials for each side; fall back to the single value passed in
    left_vals  = np.asarray(_fetch_side_vals('Left')  or [left_value],  dtype=float)
    right_vals = np.asarray(_fetch_side_vals('Right') or [right_value], dtype=float)

    left_mean,  left_max  = left_vals.mean(),  left_vals.max()
    right_mean, right_max = right_vals.mean(), right_vals.max()

    left_pct  = calculate_percentile(left_mean,  reference_data)
    right_pct = calculate_percentile(right_mean, reference_data)

    # ── plot ────────────────────────────────────────────────────────────────
    plt.figure(facecolor='#181818')
    ax = plt.subplot(111, facecolor='#303030')

    ax.hist(reference_data, bins=20, color='cornflowerblue',
            alpha=0.7, edgecolor='white', label='Reference')

    ax.axvline(left_value,  color='green',  ls='--', lw=2, label='Left (latest)')
    ax.axvline(right_value, color='orange', ls='--', lw=2, label='Right (latest)')

    ax.set_xlabel(variable.replace('_', ' '), color='slategrey')
    ax.set_ylabel('Frequency',               color='slategrey')
    ax.tick_params(axis='x', colors='lightgrey')
    ax.tick_params(axis='y', colors='lightgrey')
    ax.grid(color='dimgrey')

    txt = (
        f'LEFT  – mean: {left_mean:.2f}\n'
        f'        max:  {left_max:.2f}\n'
        f'        %ile: {left_pct:.1f}\n'
        f'RIGHT – mean: {right_mean:.2f}\n'
        f'        max:  {right_max:.2f}\n'
        f'        %ile: {right_pct:.1f}'
    )
    plt.text(0.95, 0.05, txt, ha='right', va='bottom',
             transform=ax.transAxes, color='white', fontsize=9,
             backgroundcolor='#181818')

    ax.legend(facecolor='black', edgecolor='grey',
              prop={'size': 'small'}, labelcolor='grey')

    out_path = os.path.join(tmpdirname, f'{variable}_histogram_slv.png')
    plt.savefig(out_path, bbox_inches='tight', facecolor='#181818')
    plt.close()
    return out_path

# Prepare the document
doc = Document()
doc.add_picture("8ctane Baseball - Black abd Blue BG.jpeg", width=Inches(4.0))  # Replace with your logo path
doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER

# Adding player name and date
doc.add_paragraph(f"Player's Name: {client_name}")  # Replace client_name with dynamic value
doc.add_paragraph(f"Date: {date.today().strftime('%B %d, %Y')}")

# Create a temporary directory to store images
with tempfile.TemporaryDirectory() as tmpdirname:
    # List of movements to process
    movements = ['CMJ', 'DJ', 'PPU', 'SLV', 'NMT']
    
    for movement in movements:
        # Add movement title
        doc.add_paragraph(f"{movement} Report", style='Title')
        doc.add_paragraph(f"This section includes percentile reports and comparisons for {movement}.", style='Heading 2')

        if movement == 'CMJ':
            # Fetch CMJ data for the client
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM CMJ WHERE name = ?
            """, (client_name,))
            client_cmj_data = client_cursor.fetchone()
            reference_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM CMJ
            """)
            reference_cmj_data = np.array(reference_cursor.fetchall())
            
            # --- CMJ power files & analysis ---
            cmj_files = find_power_files("CMJ")
            if cmj_files:
                cmj_traces = [load_power_txt(pf) for pf in cmj_files]
                add_power_analysis_section(
                    doc, movement="CMJ", traces=cmj_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="CMJ"
                )

            # Ensure data exists before proceeding
            if client_cmj_data and reference_cmj_data.size > 0:
                # Generate bar graphs for each variable in CMJ
                variables = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
                for i, var in enumerate(variables):
                    # Format the variable name by removing underscores
                    formatted_var = var.replace('_', ' ')
                    
                    # Add variable title before the graph
                    doc.add_paragraph(f"{formatted_var} Comparison", style='Heading 2')
                    
                    # Generate the bar graph and add to document
                    bar_image = generate_bar_graph(var, client_cmj_data[i], reference_cmj_data[:, i], f'{formatted_var} Comparison', tmpdirname)
                    doc.add_picture(bar_image, width=Inches(6))
                
                # Generate scatter plot for CMJ (Force_Peak_Power vs. Velo_Peak_Power)
                client_cmj_dict = {'Force_at_PP': client_cmj_data[3],
                   'Vel_at_PP':   client_cmj_data[4]}
                reference_cmj_dict = pd.DataFrame(reference_cmj_data, columns=variables)
                
                # Add scatter plot title and image
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                scatter_image = generate_scatter_plot(client_cmj_dict, reference_cmj_dict,
                                                      'Force_at_PP', 'Vel_at_PP',
                                                      'CMJ: Force vs. Velocity', tmpdirname)
                doc.add_picture(scatter_image, width=Inches(6))
    
        # ───────────────────────── DJ ─────────────────────────        
        elif movement == 'DJ':
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP, CT, RSI
                FROM DJ
                WHERE name = ?
            """, (client_name,))
            client_dj_data = client_cursor.fetchone()

            # ── REFERENCE (robust: pandas + print counts) ─────────────────────
            dj_vars = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg',
                       'Force_at_PP', 'Vel_at_PP', 'CT', 'RSI']
            ref_sql = f"SELECT {', '.join(dj_vars)} FROM DJ"
            reference_dj_df = pd.read_sql_query(ref_sql, reference_conn)
            print(f"DJ reference rows: {len(reference_dj_df)}")
            reference_dj_data = reference_dj_df.to_numpy(dtype=float, copy=False)

            # ── POWER OVERLAY (files like *DJ*_Power*.txt or DJ_Power*.txt) ───
            power_dir = r"D:\Athletic Screen 2.0\Output Files"
            dj_power_files = (sorted(globmod.glob(os.path.join(power_dir, "*DJ*_Power*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "DJ_Power*.txt"))))
            # --- DJ power files & analysis ---
            dj_files = find_power_files("DJ")
            if dj_files:
                dj_traces = [load_power_txt(pf) for pf in dj_files]
                add_power_analysis_section(
                    doc, movement="DJ", traces=dj_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="DJ"
                )

            # ── BAR GRAPHS + SCATTER (with reference) ─────────────────────────
            if client_dj_data:
                for i, var in enumerate(dj_vars):
                    doc.add_paragraph(f"{var.replace('_',' ')} Comparison", style='Heading 2')
                    ref_col = reference_dj_data[:, i] if len(reference_dj_df) else np.array([])
                    bar = generate_bar_graph(var, client_dj_data[i], ref_col,
                                             f'{var} Comparison', tmpdirname)
                    doc.add_picture(bar, width=Inches(6))

                client_dj_dict = {'Force_at_PP': client_dj_data[3],
                                  'Vel_at_PP'  : client_dj_data[4]}
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                dj_scatter = generate_scatter_plot(
                    client_dj_dict,
                    reference_dj_df if not reference_dj_df.empty else pd.DataFrame(columns=dj_vars),
                    'Force_at_PP', 'Vel_at_PP',
                    'DJ: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(dj_scatter, width=Inches(6))
            else:
                print("No DJ client row found.")
                
        # ───────────────────────── PPU ───────────────────────   
        elif movement == 'PPU':
            # Fetch PPU data for the client
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM PPU WHERE name = ?
            """, (client_name,))
            client_ppu_data = client_cursor.fetchone()

            # Reference pull
            reference_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM PPU
            """)
            reference_ppu_data = np.array(reference_cursor.fetchall(), dtype=float)

            # --- PPU power files & analysis (optional) ---
            ppu_files = find_power_files("PPU")
            if ppu_files:
                ppu_traces = [load_power_txt(pf) for pf in ppu_files]
                add_power_analysis_section(
                    doc, movement="PPU", traces=ppu_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="PPU"
                )

            # Ensure data exists before proceeding
            if client_ppu_data is not None and reference_ppu_data.size > 0:
                variables = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']

                # Percentile histograms (same as CMJ)
                for i, var in enumerate(variables):
                    formatted_var = var.replace('_', ' ')
                    doc.add_paragraph(f"{formatted_var} Comparison", style='Heading 2')
                    bar_image = generate_bar_graph(
                        var,
                        float(client_ppu_data[i]),
                        reference_ppu_data[:, i],
                        f'{formatted_var} Comparison',
                        tmpdirname
                    )
                    doc.add_picture(bar_image, width=Inches(6))

                # Force–Velocity scatter (same as CMJ)
                client_ppu_dict = {
                    'Force_at_PP': float(client_ppu_data[3]),
                    'Vel_at_PP'  : float(client_ppu_data[4]),
                }
                reference_ppu_df = pd.DataFrame(reference_ppu_data, columns=variables)
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                ppu_scatter = generate_scatter_plot(
                    client_ppu_dict, reference_ppu_df,
                    'Force_at_PP', 'Vel_at_PP',
                    'PPU: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(ppu_scatter, width=Inches(6))
            else:
                print("PPU: missing client or reference data; skipping figure set.")

        # ───────────────────────── SLV ───────────────────────        
        elif movement == 'SLV':
            # ── CLIENT ROWS (Left/Right) ───────────────────────────────────────
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP
                FROM SLV WHERE name = ? AND side = 'Left'
            """, (client_name,))
            client_slvl_data = client_cursor.fetchone()

            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP
                FROM SLV WHERE name = ? AND side = 'Right'
            """, (client_name,))
            client_slvr_data = client_cursor.fetchone()

            # ── REFERENCE (keep side to filter/inspect if needed) ─────────────
            slv_vars_no_side = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
            ref_sql = f"SELECT side, {', '.join(slv_vars_no_side)} FROM SLV"
            reference_slv_df = pd.read_sql_query(ref_sql, reference_conn)
            print(f"SLV reference rows: {len(reference_slv_df)}")

            # ── POWER OVERLAYS (Left & Right) ─────────────────────────────────
            # We try common patterns. Adjust to your actual export names if needed.
            power_dir = r"D:\Athletic Screen 2.0\Output Files"

            # Left trials
            slv_left_files = (sorted(globmod.glob(os.path.join(power_dir, "*SLVL*_Power*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "SLV_Power_Left*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "SLV_Left*_Power*.txt"))))

            # Right trials
            slv_right_files = (sorted(globmod.glob(os.path.join(power_dir, "*SLVR*_Power*.txt"))) or
                   sorted(globmod.glob(os.path.join(power_dir, "SLV_Power_Right*.txt"))) or
                   sorted(globmod.glob(os.path.join(power_dir, "SLV_Right*_Power*.txt"))))

            # --- SLV power files & analysis ---
            slv_files = find_power_files("SLV")
            if slv_files:
                slv_traces = [load_power_txt(pf) for pf in slv_files]
                add_power_analysis_section(
                    doc, movement="SLV", traces=slv_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="SLV"
                )

            # ── HISTOGRAMS + SCATTER (with reference) ─────────────────────────
            if client_slvl_data and client_slvr_data and not reference_slv_df.empty:
                # Build name→value dicts so we don't rely on fragile positional indexes
                slv_cols = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
                client_slvl = dict(zip(slv_cols, map(float, client_slvl_data)))
                client_slvr = dict(zip(slv_cols, map(float, client_slvr_data)))
            
                # We want JH, PP, Force@PP (fixed), and Vel@PP
                for var in ['JH_IN', 'PP_FORCEPLATE', 'Force_at_PP', 'Vel_at_PP']:
                    doc.add_paragraph(f"{var.replace('_',' ')} Comparison (Left vs Right)",
                                      style='Heading 2')
            
                    # Reference distribution by column name
                    ref_col = reference_slv_df[var].to_numpy(dtype=float, copy=False)
            
                    # Client values by column name (correct variables now)
                    left_val  = client_slvl.get(var, np.nan)
                    right_val = client_slvr.get(var, np.nan)
            
                    hist = generate_slv_histogram(
                        var,
                        left_val,            # Left
                        right_val,           # Right
                        ref_col,
                        f'{var} Comparison',
                        tmpdirname
                    )
                    doc.add_picture(hist, width=Inches(6))
            
                # Scatter Force vs Velocity (merge both sides)
                client_slv_all = np.array([list(client_slvl.values()),
                                           list(client_slvr.values())], dtype=float)
                client_slv_dict = {
                    'Force_at_PP': [client_slvl['Force_at_PP'], client_slvr['Force_at_PP']],
                    'Vel_at_PP'  : [client_slvl['Vel_at_PP'],   client_slvr['Vel_at_PP']],
                }
                ref_for_scatter = reference_slv_df[['Force_at_PP', 'Vel_at_PP']]
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                slv_scatter = generate_scatter_plot(
                    client_slv_dict,
                    ref_for_scatter,
                    'Force_at_PP', 'Vel_at_PP',
                    'SLV: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(slv_scatter, width=Inches(6))
            else:
                if not (client_slvl_data and client_slvr_data):
                    print("⚠️  Missing SLV client data – skipping SLV graphs.")
                if reference_slv_df.empty:
                    print("⚠️  SLV reference pull returned 0 rows.")
    
        # elif movement == 'NMT':
        #     # Fetch NMT data for the client (10s taps only)
        #     print("Entering NMT section…")
        #     client_cursor.execute("SELECT NUM_TAPS_10s FROM NMT WHERE name = ?", (client_name,))
        #     client_nmt_data = client_cursor.fetchone()
        # 
        #     reference_cursor.execute("SELECT NUM_TAPS_10s FROM NMT")
        #     reference_nmt_data = np.array(reference_cursor.fetchall(), dtype=float)
        # 
        #     print(f"NMT client row present: {client_nmt_data is not None}, reference rows: {len(reference_nmt_data)}")
        # 
        #     if client_nmt_data and reference_nmt_data.size > 0:
        #         nmt_var_label = 'NUM TAPS (10s)'
        #         doc.add_paragraph(f"{nmt_var_label} Comparison", style='Heading 2')
        # 
        #         # Pass table_hint="NMT" to avoid mis-inference
        #         nmt_image = generate_bar_graph(
        #             'NUM_TAPS_10s',
        #             float(client_nmt_data[0]),
        #             reference_nmt_data[:, 0],
        #             f'{nmt_var_label} Comparison',
        #             tmpdirname          # ← stop here
        #         )
        #         doc.add_picture(nmt_image, width=Inches(6))
        #     else:
        #         print("NMT: missing client or reference data; skipping figure.")


# Function to convert DOCX to images
def docx_to_images(docx_path, output_dir):
    # Extract text from the DOCX file
    text = docx2txt.process(docx_path)
    
    # Split the text into lines
    lines = text.splitlines()

    # Create a blank image with white background
    img_width, img_height = 1000, 1500
    image = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(image)

    # Use a simple font
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Draw the text onto the image
    padding = 20
    y_text = padding
    for line in lines:
        if y_text + padding > img_height:
            # Save the image and start a new one if the text exceeds the page height
            img_path = os.path.join(output_dir, f"page_{int(y_text / img_height)}.png")
            image.save(img_path)
            y_text = padding
            image = Image.new('RGB', (img_width, img_height), color='white')
            draw = ImageDraw.Draw(image)

        # Calculate text size and draw it
        text_bbox = draw.textbbox((0, 0), line, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        draw.text((padding, y_text), line, font=font, fill="black")
        y_text += text_height + padding

    # Save the last image
    img_path = os.path.join(output_dir, "final_page.png")
    image.save(img_path)

    return img_path

doc.save(output_filename)          # ← only one final save
print(f"Document saved at: {output_filename}")

# Close connections
client_conn.close()
reference_conn.close()

# Example usage
img_output_directory = r'G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Images'
os.makedirs(img_output_directory, exist_ok=True)

# Convert DOCX to images
img_path = docx_to_images(output_filename, img_output_directory)
print(f"Images saved at {img_path}")

Databases opened successfully.
Client Name: Jordan Driver
DJ reference rows: 73
SLV reference rows: 100
Document saved at: G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Athletic_Report_Jordan Driver_2025-11-07.docx
Images saved at G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Images\final_page.png


In [3]:
# Creates full report for All group comparison

import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import date
import tempfile
import docx2txt
from PIL import Image, ImageDraw, ImageFont
import os, re
from scipy import integrate, stats
import glob as globmod

# -------- style to match your dark report --------
plt.rcParams.update({
    "figure.facecolor": "#181818",
    "axes.facecolor"  : "#303030",
    "axes.edgecolor"  : "white",
    "axes.labelcolor" : "slategrey",
    "xtick.color"     : "lightgrey",
    "ytick.color"     : "lightgrey",
    "grid.color"      : "dimgrey",
    "text.color"      : "white",
})
# Corrected file paths with raw strings to handle backslashes properly
client_db_path = r'D:\Athletic Screen 2.0\Output Files\movement_database_v2.db'
reference_db_path = r'D:\Athletic Screen 2.0\Output Files\Athletic_Screen_All_data_v2.db'

# Ensure the paths are valid and accessible
if not os.path.exists(client_db_path):
    print(f"Client database not found at {client_db_path}")
if not os.path.exists(reference_db_path):
    print(f"Reference database not found at {reference_db_path}")

# Connect to the client and reference databases
client_conn = sqlite3.connect(client_db_path)
reference_conn = sqlite3.connect(reference_db_path)
client_cursor = client_conn.cursor()
reference_cursor = reference_conn.cursor()

print("Databases opened successfully.")

# Fetch the client's name from the database (assuming the 'name' column is in all tables)
client_cursor.execute("SELECT DISTINCT name FROM CMJ")  # Change table if necessary
client_name = client_cursor.fetchone()[0]  # Get the first row and first column
print(f"Client Name: {client_name}")

# ---------- build unique export paths (date-stamped, no overwrite) -----
client_cursor.execute("SELECT MAX(date) FROM CMJ WHERE name = ?", (client_name,))
assessment_date = client_cursor.fetchone()[0]          # e.g. '2025-05-22'
if not assessment_date:                                # fallback to today
    assessment_date = date.today().strftime("%Y-%m-%d")

reports_dir = r'G:\My Drive\Athletic Screen 2.0 Reports\College Reports'
os.makedirs(reports_dir, exist_ok=True)

parts = client_name.split(', ')
client_name_rev = f"{parts[1]}_{parts[0]}" if len(parts) == 2 else client_name
base_name = f"Athletic_Report_{client_name_rev}_{assessment_date}_All"

output_filename = os.path.join(reports_dir, base_name + ".docx")
img_output_directory = os.path.join(reports_dir, "Images", base_name)
os.makedirs(img_output_directory, exist_ok=True)

# auto-increment if the same file already exists
counter = 1
while os.path.exists(output_filename):
    output_filename = os.path.join(
        reports_dir, f"{base_name}_{counter}.docx"
    )
    img_output_directory = os.path.join(
        reports_dir, "Images", f"{base_name}_{counter}"
    )
    os.makedirs(img_output_directory, exist_ok=True)
    counter += 1
# ----------------------------------------------------------------------

# Helper function to calculate percentile
def calculate_percentile(value, reference_data):
    return stats.percentileofscore(reference_data, value)

# ─── UPDATED generate_bar_graph ────────────────────────────────────────────────
def generate_bar_graph(variable, client_value, reference_data, title, tmpdirname):
    """
    Blue bars  = reference distribution
    ─ red      = client MAX (best trial **within the same movement table**)
    ─ violet   = client MEAN (average of those trials)

    • No code outside this function needs to change.
    • If the caller still passes one score, that is fine; this function looks
      up any matching trials on the same assessment day and combines them.
    • RSI histograms use 0.25-wide bins so you see bars at 1.25, 1.50, 1.75 …
    """
    import numpy as np, matplotlib.pyplot as plt, os, sqlite3, re

    # ───────────── lookup: which movement table are we dealing with? ─────────
    column_table_map = {"CMJ": ["CMJ"], "DJ": ["DJ"], "PPU": ["PPU"], "SLV": ["SLV"], "NMT": ["NMT"]}
    table_guess = None
    for tbl in column_table_map:          # CMJ, DJ, SLV, NMT
        try:
            reference_cursor.execute(f"SELECT COUNT(*) FROM {tbl}")
            if reference_cursor.fetchone()[0] == len(reference_data):
                table_guess = tbl
                break
        except sqlite3.OperationalError:
            continue
    if table_guess is None:               # fallback
        for tbl in column_table_map:
            try:
                client_cursor.execute(f"SELECT 1 FROM {tbl} LIMIT 1")
                table_guess = tbl
                break
            except sqlite3.OperationalError:
                continue

    # ───────────── gather all trials for this athlete / table / day ──────────
    scores = []
    if table_guess:
        # Some tables (e.g., NMT) may not have trial_name. If so, fall back safely.
        try:
            client_cursor.execute(
                f"SELECT trial_name FROM {table_guess} WHERE name=? LIMIT 1",
                (client_name,),
            )
            row = client_cursor.fetchone()
            date_prefix = None
            if row and row[0]:
                m = re.match(r"(\d{4}[-_]\d{2}[-_]\d{2})", row[0])
                date_prefix = m.group(1) if m else None
    
            if date_prefix:
                q = f"SELECT {variable} FROM {table_guess} WHERE name=? AND trial_name LIKE ?"
                client_cursor.execute(q, (client_name, f"{date_prefix}%"))
            else:
                q = f"SELECT {variable} FROM {table_guess} WHERE name=?"
                client_cursor.execute(q, (client_name,))
    
            scores = [r[0] for r in client_cursor.fetchall() if r[0] is not None]
        except sqlite3.OperationalError:
            # No trial_name column (likely NMT) → just use the single value
            scores = [client_value]
    
    # fallback if still empty
    if not scores:
        scores = [client_value]


    scores = np.asarray(scores, dtype=float)
    c_mean = scores.mean()
    
    # --- choose the extreme we draw as the red line ---------------------------
    if variable.upper() == "CT":        # Contact-Time → use the LOWEST value
        c_extreme = scores.min()
        extreme_label = "Client Min"
    else:                               # every other metric → highest value
        c_extreme = scores.max()
        extreme_label = "Client Max"

    perc_mean = calculate_percentile(c_mean, reference_data)

    # ──────────────────────────── plotting begins ────────────────────────────
    plt.figure(facecolor="#181818")
    ax = plt.subplot(111, facecolor="#303030")

    reference_plotted = False  # flag to avoid double-plotting

    # ---------- RSI special case: 0.25-wide bars & custom ticks -------------
    if variable.upper() == "RSI":
        lo = np.floor(reference_data.min() / 0.25) * 0.25
        hi = np.ceil(reference_data.max() / 0.25) * 0.25
        bins = np.arange(lo, hi + 0.25, 0.25)      # bin edges
        centers = bins[:-1]                         # bar positions

        counts, _ = np.histogram(reference_data, bins=bins)
        ax.bar(
            centers,
            counts,
            width=0.25,
            align="edge",
            color="cornflowerblue",
            alpha=0.7,
            edgecolor="white",
            label="Reference",
        )
        # after counts, _ = np.histogram(...)
        for x, h in zip(centers, counts):
            if h == 0:                        # empty bin → draw thin outline
                ax.bar(x, 1e-6, width=.25, align='edge',
                       color='none', edgecolor='#404040', linewidth=.5)

        ax.set_xticks(centers)
        ax.set_xticklabels([f"{x:.2f}" for x in centers], color="lightgrey")

        reference_plotted = True  # we already drew the reference bars
    else:
        bins = 20  # default bin count

    # ---------- draw reference histogram when not plotted above -------------
    if not reference_plotted:
        ax.hist(
            reference_data,
            bins=bins,
            color="cornflowerblue",
            alpha=0.7,
            edgecolor="white",
            label="Reference",
        )

    # ---------- client mean / max lines -------------------------------------
    ax.axvline(c_extreme, color="red", ls="--", lw=2, label="Client Max")
    ax.axvline(c_mean, color="violet", ls="--", lw=2, label="Client Mean")

    # ---------- cosmetics ----------------------------------------------------
    ax.set_xlabel(variable.replace("_", " "), color="slategrey")
    ax.set_ylabel("Frequency", color="slategrey")
    ax.tick_params(axis="x", colors="lightgrey")
    ax.tick_params(axis="y", colors="lightgrey")
    ax.grid(color="dimgrey")

    txt = (
        f"Percentile (mean): {perc_mean:.1f}%"
        f"\nMean: {c_mean:.2f}"
        f"\nMax:  {c_extreme:.2f}"
    )
    plt.text(
        0.95,
        0.05,
        txt,
        ha="right",
        va="bottom",
        transform=ax.transAxes,
        color="white",
        fontsize=9,
        backgroundcolor="#181818",
    )

    ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")

    # ---------- save ---------------------------------------------------------
    out_path = os.path.join(tmpdirname, f"{variable}_histogram.png")
    plt.savefig(out_path, bbox_inches="tight", facecolor="#181818")
    plt.close()
    return out_path

# Function to generate scatter plot for CMJ
def generate_scatter_plot(client_data, reference_data, x_var, y_var, title, tmpdirname):
    plt.figure(facecolor='#181818', figsize=(6, 6))
    ax = plt.subplot(111, facecolor='#303030')

    # Create scatter plot for reference data (cornflower blue)
    ax.scatter(reference_data[x_var], reference_data[y_var], label='Reference', alpha=0.5, color='cornflowerblue')

    # Create scatter plot for client data (red)
    ax.scatter(client_data[x_var], client_data[y_var], label='Client', color='red', edgecolors='black', s=100)

    # Set axis labels, replacing underscores with spaces
    ax.set_xlabel(x_var.replace('_', ' '), color='slategrey')
    ax.set_ylabel(y_var.replace('_', ' '), color='slategrey')

    # Dynamically set ticks and numbers to light grey
    ax.tick_params(axis='x', colors='lightgrey')
    ax.tick_params(axis='y', colors='lightgrey')

    # Add vertical and horizontal reference lines (light grey)
    ax.axvline(x=np.mean(reference_data[x_var]), color='lightgrey', linestyle='--', linewidth=1)
    ax.axhline(y=np.mean(reference_data[y_var]), color='lightgrey', linestyle='--', linewidth=1)

    # Customize grid style
    ax.grid(color='dimgrey')

    # Add legend
    ax.legend(facecolor='black', edgecolor='grey', prop={'size': 'small'}, labelcolor='grey')

    # Save scatter plot to file
    scatter_filename = os.path.join(tmpdirname, 'cmj_scatter.png')
    plt.savefig(scatter_filename, bbox_inches='tight', facecolor='#181818')
    plt.close()

    return scatter_filename


def load_power_txt(txt_path: str) -> pd.Series:
    """
    Parse exported power file (like your example). Returns a pandas Series of power.
    Assumes: header lines, then a line starting with 'ITEM', then data rows:
             <index>\t<value>
    Skips rows without a numeric second field.
    """
    power_vals = []
    in_data = False
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        for raw in f:
            line = raw.strip()
            if not in_data:
                if line.startswith("ITEM"):
                    in_data = True
                continue
            # from here on, try to parse the last field as a float
            if not line:
                continue
            parts = re.split(r"\t+", line)
            if len(parts) < 2:
                # sometimes a blank power cell exists on the first data row — skip it
                continue
            try:
                val = float(parts[-1])
                power_vals.append(val)
            except ValueError:
                # non-numeric tail → ignore
                continue
    if not power_vals:
        raise ValueError(f"No power values parsed from {txt_path}")
    return pd.Series(power_vals, name="Power")

def analyze_power_curve(power: pd.Series, fs_hz: float = 1000.0) -> dict:
    """
    Compute useful shape/temporal features.
    power: Series of power (W). fs_hz: sampling rate (Hz). If unknown, 1000 Hz is typical.
    Returns a dict of metrics.
    """
    p = np.asarray(power, dtype=float)
    n = p.size
    t = np.arange(n) / fs_hz

    # basic
    p_peak_idx = int(np.nanargmax(p))
    p_peak     = float(p[p_peak_idx])
    t_peak     = float(t[p_peak_idx])

    # onset/offset via % of peak (robust to baseline drift)
    thr10 = 0.10 * p_peak
    thr50 = 0.50 * p_peak
    thr90 = 0.90 * p_peak

    # first index above 10% of peak
    try:
        onset_idx = int(np.argmax(p >= thr10))
    except ValueError:
        onset_idx = 0
    # first index after peak that falls below 10% (or end)
    post = p[p_peak_idx:]
    off_rel = np.argmax(post < thr10) if np.any(post < thr10) else (post.size - 1)
    offset_idx = p_peak_idx + int(off_rel)

    # 10–90% rise time on rising limb
    rising = p[:p_peak_idx+1]
    try:
        i10 = int(np.argmax(rising >= thr10))
        i90 = int(np.argmax(rising >= thr90))
        rise_time = (i90 - i10) / fs_hz if i90 > i10 else np.nan
        rise_slope = (0.8 * p_peak) / rise_time if rise_time and rise_time > 0 else np.nan
    except ValueError:
        i10 = i90 = None
        rise_time = np.nan
        rise_slope = np.nan

    # FWHM (50% of peak) width
    # left crossing
    try:
        left_idx  = int(np.argmax(rising >= thr50))
    except ValueError:
        left_idx = p_peak_idx
    # right crossing
    falling = p[p_peak_idx:]
    try:
        right_rel = int(np.argmax(falling <= thr50))
        right_idx = p_peak_idx + right_rel
    except ValueError:
        right_idx = p_peak_idx
    fwhm_sec = (right_idx - left_idx) / fs_hz if right_idx > left_idx else np.nan

    # Work/impulse of power (area under curve) over the active window
    a = max(0, onset_idx)
    b = min(n - 1, max(offset_idx, p_peak_idx))
    auc_joules = float(np.trapezoid(np.nan_to_num(p[a:b+1], nan=0.0), dx=1.0/fs_hz))

    # timing “balance”: center of mass of power curve (0..1)
    # (earlier vs. later power concentration)
    weights = p[a:b+1].clip(min=0)
    if weights.sum() > 0:
        t_window = t[a:b+1]
        t_com = float(np.sum(t_window * weights) / np.sum(weights))
        t_com_norm = (t_com - t[a]) / max(1e-9, (t[b] - t[a]))
    else:
        t_com = np.nan
        t_com_norm = np.nan

    # variability around peak (local coefficient of variation in ±50 ms)
    w = int(0.05 * fs_hz)
    lo = max(0, p_peak_idx - w)
    hi = min(n, p_peak_idx + w + 1)
    local = p[lo:hi]
    cv_local = float(np.std(local) / np.mean(local)) if np.mean(local) > 0 else np.nan

    return {
        "n_samples": n,
        "fs_hz": fs_hz,
        "peak_power_w": p_peak,
        "time_to_peak_s": t_peak,
        "rise_time_10_90_s": float(rise_time),
        "rise_slope_w_per_s": float(rise_slope),
        "fwhm_s": float(fwhm_sec),
        "auc_j": auc_joules,
        "onset_idx": a,
        "offset_idx": b,
        "peak_idx": p_peak_idx,
        "t_com_s": t_com,
        "t_com_norm_0to1": t_com_norm,
        "cv_local_peak": cv_local,
        "i10_idx": int(i10) if isinstance(i10, int) else None,
        "i90_idx": int(i90) if isinstance(i90, int) else None,
        "left50_idx": left_idx,
        "right50_idx": right_idx,
    }

def plot_power_curve(power: pd.Series,
                     metrics: dict,
                     out_path: str,
                     title: str = "Power Curve",
                     annotate: bool = True):
    """
    Plot power vs. time with annotations (peak, 10–90 rise, FWHM, AUC window).
    """
    p = np.asarray(power, dtype=float)
    t = np.arange(p.size) / metrics["fs_hz"]

    fig, ax = plt.subplots(figsize=(6, 3.6))
    ax.plot(t, p, lw=2, label="Power")
    ax.grid(True)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Power (W)")
    ax.set_title(title, pad=8, color="white")

    # annotate regions
    a, b = metrics["onset_idx"], metrics["offset_idx"]
    ax.axvspan(t[a], t[b], color="white", alpha=0.07, label="active window")

    # peak
    pk = metrics["peak_idx"]
    ax.plot([t[pk]], [p[pk]], "o", ms=6, color="tomato", label="Peak")
    ax.axhline(p[pk]*0.5, ls="--", lw=1, color="grey")
    ax.vlines([t[metrics["left50_idx"]], t[metrics["right50_idx"]]],
              ymin=0, ymax=p[pk]*0.5, linestyles="--", colors="grey", lw=1)

    # 10–90 rise (if available)
    if metrics["i10_idx"] is not None and metrics["i90_idx"] is not None:
        ax.plot([t[metrics["i10_idx"]], t[metrics["i90_idx"]]],
                [p[metrics["i10_idx"]], p[metrics["i90_idx"]]],
                lw=3, color="deepskyblue", label="10–90% rise")

    if annotate:
        txt = (f"Peak: {metrics['peak_power_w']:.1f} W @ {metrics['time_to_peak_s']:.3f} s"
               f"\nRise 10–90: {metrics['rise_time_10_90_s']:.3f} s"
               f"\nFWHM: {metrics['fwhm_s']:.3f} s"
               f"\nWork (AUC): {metrics['auc_j']:.1f} J"
               f"\nTiming COM: {metrics['t_com_norm_0to1']:.2f} (0 early…1 late)")
        ax.text(0.99, 0.02, txt, ha="right", va="bottom",
                transform=ax.transAxes, fontsize=9, color="white",
                bbox=dict(boxstyle="round,pad=0.25", facecolor="#181818", edgecolor="#444"))

    ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

def overlay_power_trials(traces: list[pd.Series],
                         fs_hz: float,
                         out_path: str,
                         title: str = "Power (all trials)",
                         align: str = "peak",          # "peak", "onset10", or "none"
                         window_s: tuple[float, float] | None = (0.30, 0.40),
                         show_mean: bool = True):
    """
    Overlay multiple power traces and align them in time.

    align:
      - "peak"    → align each trial's max power to t=0
      - "onset10" → align first sample ≥ 10% of that trial's peak to t=0
      - "none"    → no alignment; left edges at t=0

    window_s: (pre, post) seconds to show around t=0 (None to show full extent)
    """

    # --- convert input to arrays; find alignment index per trial --------------
    arrs = [np.asarray(s, dtype=float) for s in traces if len(s) > 0]
    if not arrs:
        raise ValueError("overlay_power_trials: no non-empty traces provided")

    def _align_index(x: np.ndarray) -> int:
        if align == "none":
            return 0
        # guard against NaNs
        if not np.any(np.isfinite(x)):
            return 0
        # peak index
        try:
            pk = int(np.nanargmax(x))
        except ValueError:
            pk = 0
        if align == "peak":
            return pk
        elif align == "onset10":
            thr = 0.10 * (x[pk] if np.isfinite(x[pk]) else np.nanmax(x))
            # first index >= 10% of that trial's peak
            idx = int(np.argmax(x >= thr)) if np.any(x >= thr) else 0
            return idx
        else:
            return 0

    align_idx = [ _align_index(x) for x in arrs ]

    # --- build a padded matrix so all alignment points land at the same column
    max_left  = max(align_idx)                                  # largest left padding needed
    right_len = [len(x) - i for x, i in zip(arrs, align_idx)]   # samples from align idx to end
    max_right = max(right_len)
    L = max_left + max_right                                     # total aligned length

    aligned = np.full((len(arrs), L), np.nan)
    for r, (x, i0) in enumerate(zip(arrs, align_idx)):
        start = max_left - i0
        aligned[r, start:start+len(x)] = x

    # --- time vector: t=0 at the common alignment column ---------------------
    t = (np.arange(L) - max_left) / fs_hz

    # --- optional cropping around t=0 ----------------------------------------
    if window_s is not None:
        pre, post = window_s
        i_lo = max(0, int(np.floor((-pre)  * fs_hz)) + max_left)
        i_hi = min(L, int(np.ceil( (post) * fs_hz)) + max_left)
        aligned = aligned[:, i_lo:i_hi]
        t = t[i_lo:i_hi]
        
    # --- plot ----------------------------------------------------------------
    fig, ax = plt.subplots(figsize=(6, 3.6))
    
    # consistent y-limits (same scale across trials)
    y_max = np.nanmax(aligned)
    y_min = np.nanmin(aligned)
    
    # NEW: mask-based plotting so left/right NaNs don't truncate the curve visually
    coverage_flags = []  # we'll use this to warn about very short traces
    for i in range(aligned.shape[0]):
        row = aligned[i]
        mask = np.isfinite(row)
        cov = np.count_nonzero(mask) / max(1, len(row))
        coverage_flags.append(cov)
        if np.count_nonzero(mask) >= 2:
            ax.plot(t[mask], row[mask], lw=1.2, alpha=0.6)
    
    # optional: highlight which trials are very short (<50% of the window)
    short = [idx for idx, c in enumerate(coverage_flags, start=1) if c < 0.5]
    if short:
        print(f"⚠️ overlay_power_trials: {len(short)} trial(s) with <50% coverage in the window: {short}. "
              f"Consider changing window_s or checking the export files.")
    
    if show_mean:
        mean_curve = np.nanmean(aligned, axis=0)
        ax.plot(t, mean_curve, lw=2.2, color="cyan", label="Mean")
    
    # vertical line at alignment point
    ax.axvline(0.0, color="grey", lw=1, ls="--",
               label=("Aligned peak" if align == "peak" else "Aligned onset"))
    
    ax.grid(True)
    ax.set_xlabel("Time (s, aligned)")
    ax.set_ylabel("Power (W)")
    ax.set_title(title, pad=8, color="white")
    if show_mean:
        ax.legend(facecolor="black", edgecolor="grey", prop={"size": "small"}, labelcolor="grey")
    
    ax.set_ylim(y_min - 0.05 * abs(y_max - y_min), y_max + 0.05 * abs(y_max - y_min))
    
    fig.tight_layout()
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

def find_power_files(movement: str,
                     base_dir: str = r'D:\Athletic Screen 2.0\Output Files') -> list[str]:
    """
    Finds exported power files for a movement. Supports:
      CMJ_Power.txt, CMJ1_Power.txt, CMJ2_Power.txt, ...
      DJ_Power.txt,  DJ*_Power.txt
      SLV_Power.txt, SLV*_Power.txt
    Returns a sorted (unique) list of full paths.
    """
    pats = [f"{movement}_Power.txt", f"{movement}*_Power.txt"]
    files = []
    for p in pats:
        files += globmod.glob(os.path.join(base_dir, p))
    return sorted(set(files))

def build_aligned_matrix(traces: list[pd.Series],
                         fs_hz: float,
                         align: str = "peak",
                         window_s: tuple[float, float] | None = (0.30, 0.40)):
    """
    Returns (aligned 2D array [n_trials x T], time vector [T], indices_info[list]).
    Time is centered so the chosen alignment point is at t=0.
    """
    arrs = [np.asarray(s, dtype=float) for s in traces if len(s) > 0]
    if not arrs:
        raise ValueError("build_aligned_matrix: no non-empty traces")

    def _align_idx(x: np.ndarray) -> int:
        if align == "none":
            return 0
        if not np.any(np.isfinite(x)):
            return 0
        pk = int(np.nanargmax(x))
        if align == "peak":
            return pk
        elif align == "onset10":
            thr = 0.10 * (x[pk] if np.isfinite(x[pk]) else np.nanmax(x))
            return int(np.argmax(x >= thr)) if np.any(x >= thr) else 0
        return 0

    aidx = [_align_idx(x) for x in arrs]
    max_left = max(aidx)
    right_len = [len(x) - i for x, i in zip(arrs, aidx)]
    max_right = max(right_len)
    L = max_left + max_right

    aligned = np.full((len(arrs), L), np.nan)
    for r, (x, i0) in enumerate(zip(arrs, aidx)):
        start = max_left - i0
        aligned[r, start:start+len(x)] = x

    t = (np.arange(L) - max_left) / fs_hz

    if window_s is not None:
        pre, post = window_s
        i_lo = max(0, int(np.floor((-pre)  * fs_hz)) + max_left)
        i_hi = min(L, int(np.ceil( (post) * fs_hz)) + max_left)
        return aligned[:, i_lo:i_hi], t[i_lo:i_hi], aidx
    return aligned, t, aidx

def mean_aligned_curve(traces: list[pd.Series],
                       fs_hz: float,
                       align: str = "peak",
                       window_s: tuple[float, float] | None = (0.30, 0.40)) -> pd.Series:
    aligned, t, _ = build_aligned_matrix(traces, fs_hz, align, window_s)
    mean_curve = np.nanmean(aligned, axis=0)
    return pd.Series(mean_curve, index=t, name="Power")

def analyze_power_curve_advanced(power: pd.Series, fs_hz: float = 1000.0) -> dict:
    """
    Extends your analyze_power_curve with additional, practical features:
      • rpd_max (max rate of power development) & time to RPDmax
      • AUC early (pre-peak) / late (post-peak), % early work
      • decay_90_10 on falling limb
      • skewness, kurtosis
      • spectral centroid (how ‘fast’ the curve is in frequency domain)
    """
    base = analyze_power_curve(power, fs_hz)
    p = np.asarray(power, dtype=float)
    n = p.size
    t = np.arange(n) / fs_hz

    # RPD
    dp = np.gradient(p, 1.0/fs_hz)
    rpd_max = float(np.nanmax(dp))
    rpd_idx = int(np.nanargmax(dp))
    base["rpd_max_w_per_s"]   = rpd_max
    base["time_to_rpd_max_s"] = rpd_idx / fs_hz

    # Early/late work around peak (use on/peak/off from base)
    a, b, pk = base["onset_idx"], base["offset_idx"], base["peak_idx"]
    auc_pre  = float(np.trapezoid(np.nan_to_num(p[a:pk+1],  nan=0.0), dx=1.0/fs_hz)) if pk >= a else np.nan
    auc_post = float(np.trapezoid(np.nan_to_num(p[pk:b+1], nan=0.0), dx=1.0/fs_hz)) if b >= pk else np.nan
    total    = (auc_pre if np.isfinite(auc_pre) else 0) + (auc_post if np.isfinite(auc_post) else 0)
    base["auc_pre_j"]      = auc_pre
    base["auc_post_j"]     = auc_post
    base["work_early_pct"] = float(100.0 * auc_pre / total) if total > 0 else np.nan

    # Decay time 90→10% of peak on falling limb
    peak_val = p[pk]
    fall = p[pk:]
    thr90 = 0.90 * peak_val
    thr10 = 0.10 * peak_val
    i90 = int(np.argmax(fall <= thr90)) if np.any(fall <= thr90) else 0
    i10 = int(np.argmax(fall <= thr10)) if np.any(fall <= thr10) else len(fall)-1
    base["decay_90_10_s"] = (i10 - i90) / fs_hz if i10 > i90 else np.nan

    # Shape stats
    finite = np.isfinite(p)
    base["skewness"] = float(stats.skew(p[finite])) if np.any(finite) else np.nan
    base["kurtosis"] = float(stats.kurtosis(p[finite], fisher=True)) if np.any(finite) else np.nan

    # Spectral centroid
    x = p - np.nanmean(p)
    X = np.abs(np.fft.rfft(np.nan_to_num(x)))
    freqs = np.fft.rfftfreq(x.size, d=1.0/fs_hz)
    base["spectral_centroid_hz"] = float(np.sum(freqs * X) / max(1e-12, np.sum(X)))

    return base

def add_power_analysis_section(doc: Document,
                               movement: str,
                               traces: list[pd.Series],
                               fs_hz: float,
                               tmpdirname: str,
                               reference_cursor,
                               reference_table: str):
    """
    1) Overlay (peaks aligned) + mean curve
    2) Annotated mean power curve
    3) Table of per-trial metrics + Mean/SD
    4) Adds reference percentile for peak power (vs PP_FORCEPLATE in ref DB)
    """
    # --- 1) Overlay (aligned at peak) ---------------------------------------
    overlay_png = os.path.join(tmpdirname, f"{movement}_power_overlay.png")
    overlay_power_trials(
        traces, fs_hz=fs_hz, out_path=overlay_png,
        title=f"{movement} Power – All Trials (peaks aligned)",
        align="peak", window_s=(0.30, 0.40), show_mean=True
    )
    doc.add_paragraph("Power Curves (aligned at peak)", style="Heading 2")
    doc.add_picture(overlay_png, width=Inches(6))

    # --- 2) Mean curve (aligned) + annotated plot ---------------------------
    mean_series = mean_aligned_curve(traces, fs_hz, align="peak", window_s=(0.30, 0.40))
    mean_metrics = analyze_power_curve_advanced(mean_series, fs_hz=fs_hz)

    mean_png = os.path.join(tmpdirname, f"{movement}_power_mean_annotated.png")
    plot_power_curve(mean_series, mean_metrics, mean_png,
                     title=f"{movement} – Mean Power (aligned)")
    doc.add_paragraph("Mean Power Curve (annotated)", style="Heading 3")
    doc.add_picture(mean_png, width=Inches(6))

    # --- 3) Per-trial metrics & summary -------------------------------------
    per = [analyze_power_curve_advanced(s, fs_hz=fs_hz) for s in traces]
    df  = pd.DataFrame(per)

    # Pick the important rows for the document (order here = row order in table)
    metric_rows = [
        ("Peak Power (W)",            "peak_power_w",        "{:.0f}"),
        ("Time to Peak (s)",          "time_to_peak_s",      "{:.3f}"),
        ("RPD max (W/s)",             "rpd_max_w_per_s",     "{:.0f}"),
        ("Time to RPD max (s)",       "time_to_rpd_max_s",   "{:.3f}"),
        ("Rise 10–90% (s)",           "rise_time_10_90_s",   "{:.3f}"),
        ("FWHM (s)",                  "fwhm_s",              "{:.3f}"),
        ("Work (AUC, J)",             "auc_j",               "{:.0f}"),
        ("Early work (%)",            "work_early_pct",      "{:.1f}"),
        ("Decay 90→10% (s)",          "decay_90_10_s",       "{:.3f}"),
        ("Timing CoM (0…1)",          "t_com_norm_0to1",     "{:.2f}"),
        ("Skewness",                  "skewness",            "{:.2f}"),
        ("Kurtosis",                  "kurtosis",            "{:.2f}"),
        ("Spectral centroid (Hz)",    "spectral_centroid_hz","{:.2f}"),
    ]

    # table: Metric | Trial1 | Trial2 | ... | Mean | SD
    tbl = doc.add_table(rows=1 + len(metric_rows), cols=2 + len(traces))
    tbl.style = "Light List" if "Light List" in [s.name for s in doc.styles] else tbl.style
    # header
    hdr = tbl.rows[0].cells
    hdr[0].text = "Metric"
    for i in range(len(traces)):
        hdr[1+i].text = f"Trial {i+1}"
    hdr[-1].text = "Mean ± SD"

    # body
    for r, (label, key, fmt) in enumerate(metric_rows, start=1):
        row_cells = tbl.rows[r].cells
        row_cells[0].text = label
        vals = df.get(key, pd.Series([np.nan]*len(traces))).values
        for i, v in enumerate(vals):
            row_cells[1+i].text = (fmt.format(v) if np.isfinite(v) else "—")
        mu = np.nanmean(vals)
        sd = np.nanstd(vals, ddof=1) if np.count_nonzero(np.isfinite(vals)) > 1 else np.nan
        row_cells[-1].text = (f"{fmt.format(mu)} ± {fmt.format(sd)}"
                              if np.isfinite(mu) and np.isfinite(sd) else
                              (fmt.format(mu) if np.isfinite(mu) else "—"))

    # small spacer
    doc.add_paragraph("")

    # --- 4) Reference percentile for peak power (vs. DB PP_FORCEPLATE) ------
    try:
        reference_cursor.execute(f"SELECT PP_FORCEPLATE FROM {reference_table} WHERE PP_FORCEPLATE IS NOT NULL")
        ref_pp = np.array([r[0] for r in reference_cursor.fetchall()], dtype=float)
    except sqlite3.OperationalError:
        ref_pp = np.array([])

    if ref_pp.size:
        # use the highest peak among trials (or mean peak if you prefer)
        trial_peaks = df["peak_power_w"].values
        best_peak   = float(np.nanmax(trial_peaks)) if trial_peaks.size else np.nan
        pctl        = percentile_vs_reference(best_peak, ref_pp) if np.isfinite(best_peak) else np.nan
        doc.add_paragraph(f"Reference percentile (peak power): {pctl:.1f}%", style="Intense Quote")

def percentile_vs_reference(value: float, reference_values: np.ndarray) -> float:
    """
    Wrapper for percentile (1..99) using scipy-like methodology.
    """
    return stats.percentileofscore(reference_values, value)

# Modified function to generate a histogram comparing left and right leg data
def generate_slv_histogram(variable, left_value, right_value,
                           reference_data, title, tmpdirname):
    """
    Blue bars  = reference distribution
    ─ green    = client LEFT (latest trial value you passed in)
    ─ orange   = client RIGHT

    Text box shows, for each side:
        • mean across all trials from the same assessment day
        • max across those trials
        • percentile of that mean vs. the reference distribution
    """
    import numpy as np, matplotlib.pyplot as plt, os, re, sqlite3

    # ── helper – get all trials for a given side on the same assessment day ──
    def _fetch_side_vals(side):
        # detect yyyy-mm-dd prefix in the first trial_name for this side
        client_cursor.execute(
            "SELECT trial_name FROM SLV WHERE name=? AND side=? LIMIT 1",
            (client_name, side)
        )
        row = client_cursor.fetchone()
        date_prefix = None
        if row and row[0]:
            m = re.match(r'(\d{4}[-_]\d{2}[-_]\d{2})', row[0])
            date_prefix = m.group(1) if m else None

        if date_prefix:
            q = f"SELECT {variable} FROM SLV WHERE name=? AND side=? AND trial_name LIKE ?"
            client_cursor.execute(q, (client_name, side, f'{date_prefix}%'))
        else:
            q = f"SELECT {variable} FROM SLV WHERE name=? AND side=?"
            client_cursor.execute(q, (client_name, side))

        return [r[0] for r in client_cursor.fetchall() if r[0] is not None]

    # pull all trials for each side; fall back to the single value passed in
    left_vals  = np.asarray(_fetch_side_vals('Left')  or [left_value],  dtype=float)
    right_vals = np.asarray(_fetch_side_vals('Right') or [right_value], dtype=float)

    left_mean,  left_max  = left_vals.mean(),  left_vals.max()
    right_mean, right_max = right_vals.mean(), right_vals.max()

    left_pct  = calculate_percentile(left_mean,  reference_data)
    right_pct = calculate_percentile(right_mean, reference_data)

    # ── plot ────────────────────────────────────────────────────────────────
    plt.figure(facecolor='#181818')
    ax = plt.subplot(111, facecolor='#303030')

    ax.hist(reference_data, bins=20, color='cornflowerblue',
            alpha=0.7, edgecolor='white', label='Reference')

    ax.axvline(left_value,  color='green',  ls='--', lw=2, label='Left (latest)')
    ax.axvline(right_value, color='orange', ls='--', lw=2, label='Right (latest)')

    ax.set_xlabel(variable.replace('_', ' '), color='slategrey')
    ax.set_ylabel('Frequency',               color='slategrey')
    ax.tick_params(axis='x', colors='lightgrey')
    ax.tick_params(axis='y', colors='lightgrey')
    ax.grid(color='dimgrey')

    txt = (
        f'LEFT  – mean: {left_mean:.2f}\n'
        f'        max:  {left_max:.2f}\n'
        f'        %ile: {left_pct:.1f}\n'
        f'RIGHT – mean: {right_mean:.2f}\n'
        f'        max:  {right_max:.2f}\n'
        f'        %ile: {right_pct:.1f}'
    )
    plt.text(0.95, 0.05, txt, ha='right', va='bottom',
             transform=ax.transAxes, color='white', fontsize=9,
             backgroundcolor='#181818')

    ax.legend(facecolor='black', edgecolor='grey',
              prop={'size': 'small'}, labelcolor='grey')

    out_path = os.path.join(tmpdirname, f'{variable}_histogram_slv.png')
    plt.savefig(out_path, bbox_inches='tight', facecolor='#181818')
    plt.close()
    return out_path

# Prepare the document
doc = Document()
doc.add_picture("8ctane Baseball - Black abd Blue BG.jpeg", width=Inches(4.0))  # Replace with your logo path
doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER

# Adding player name and date
doc.add_paragraph(f"Player's Name: {client_name}")  # Replace client_name with dynamic value
doc.add_paragraph(f"Date: {date.today().strftime('%B %d, %Y')}")

# Create a temporary directory to store images
with tempfile.TemporaryDirectory() as tmpdirname:
    # List of movements to process
    movements = ['CMJ', 'DJ', 'PPU', 'SLV', 'NMT']
    
    for movement in movements:
        # Add movement title
        doc.add_paragraph(f"{movement} Report", style='Title')
        doc.add_paragraph(f"This section includes percentile reports and comparisons for {movement}.", style='Heading 2')

        if movement == 'CMJ':
            # Fetch CMJ data for the client
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM CMJ WHERE name = ?
            """, (client_name,))
            client_cmj_data = client_cursor.fetchone()
            reference_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM CMJ
            """)
            reference_cmj_data = np.array(reference_cursor.fetchall())
            
            # --- CMJ power files & analysis ---
            cmj_files = find_power_files("CMJ")
            if cmj_files:
                cmj_traces = [load_power_txt(pf) for pf in cmj_files]
                add_power_analysis_section(
                    doc, movement="CMJ", traces=cmj_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="CMJ"
                )

            # Ensure data exists before proceeding
            if client_cmj_data and reference_cmj_data.size > 0:
                # Generate bar graphs for each variable in CMJ
                variables = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
                for i, var in enumerate(variables):
                    # Format the variable name by removing underscores
                    formatted_var = var.replace('_', ' ')
                    
                    # Add variable title before the graph
                    doc.add_paragraph(f"{formatted_var} Comparison", style='Heading 2')
                    
                    # Generate the bar graph and add to document
                    bar_image = generate_bar_graph(var, client_cmj_data[i], reference_cmj_data[:, i], f'{formatted_var} Comparison', tmpdirname)
                    doc.add_picture(bar_image, width=Inches(6))
                
                # Generate scatter plot for CMJ (Force_Peak_Power vs. Velo_Peak_Power)
                client_cmj_dict = {'Force_at_PP': client_cmj_data[3],
                   'Vel_at_PP':   client_cmj_data[4]}
                reference_cmj_dict = pd.DataFrame(reference_cmj_data, columns=variables)
                
                # Add scatter plot title and image
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                scatter_image = generate_scatter_plot(client_cmj_dict, reference_cmj_dict,
                                                      'Force_at_PP', 'Vel_at_PP',
                                                      'CMJ: Force vs. Velocity', tmpdirname)
                doc.add_picture(scatter_image, width=Inches(6))
    
        # ───────────────────────── DJ ─────────────────────────        
        elif movement == 'DJ':
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP, CT, RSI
                FROM DJ
                WHERE name = ?
            """, (client_name,))
            client_dj_data = client_cursor.fetchone()

            # ── REFERENCE (robust: pandas + print counts) ─────────────────────
            dj_vars = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg',
                       'Force_at_PP', 'Vel_at_PP', 'CT', 'RSI']
            ref_sql = f"SELECT {', '.join(dj_vars)} FROM DJ"
            reference_dj_df = pd.read_sql_query(ref_sql, reference_conn)
            print(f"DJ reference rows: {len(reference_dj_df)}")
            reference_dj_data = reference_dj_df.to_numpy(dtype=float, copy=False)

            # ── POWER OVERLAY (files like *DJ*_Power*.txt or DJ_Power*.txt) ───
            power_dir = r"D:\Athletic Screen 2.0\Output Files"
            dj_power_files = (sorted(globmod.glob(os.path.join(power_dir, "*DJ*_Power*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "DJ_Power*.txt"))))
            # --- DJ power files & analysis ---
            dj_files = find_power_files("DJ")
            if dj_files:
                dj_traces = [load_power_txt(pf) for pf in dj_files]
                add_power_analysis_section(
                    doc, movement="DJ", traces=dj_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="DJ"
                )

            # ── BAR GRAPHS + SCATTER (with reference) ─────────────────────────
            if client_dj_data:
                for i, var in enumerate(dj_vars):
                    doc.add_paragraph(f"{var.replace('_',' ')} Comparison", style='Heading 2')
                    ref_col = reference_dj_data[:, i] if len(reference_dj_df) else np.array([])
                    bar = generate_bar_graph(var, client_dj_data[i], ref_col,
                                             f'{var} Comparison', tmpdirname)
                    doc.add_picture(bar, width=Inches(6))

                client_dj_dict = {'Force_at_PP': client_dj_data[3],
                                  'Vel_at_PP'  : client_dj_data[4]}
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                dj_scatter = generate_scatter_plot(
                    client_dj_dict,
                    reference_dj_df if not reference_dj_df.empty else pd.DataFrame(columns=dj_vars),
                    'Force_at_PP', 'Vel_at_PP',
                    'DJ: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(dj_scatter, width=Inches(6))
            else:
                print("No DJ client row found.")
                
        # ───────────────────────── PPU ───────────────────────   
        elif movement == 'PPU':
            # Fetch PPU data for the client
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM PPU WHERE name = ?
            """, (client_name,))
            client_ppu_data = client_cursor.fetchone()

            # Reference pull
            reference_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg,
                       Force_at_PP, Vel_at_PP
                FROM PPU
            """)
            reference_ppu_data = np.array(reference_cursor.fetchall(), dtype=float)

            # --- PPU power files & analysis (optional) ---
            ppu_files = find_power_files("PPU")
            if ppu_files:
                ppu_traces = [load_power_txt(pf) for pf in ppu_files]
                add_power_analysis_section(
                    doc, movement="PPU", traces=ppu_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="PPU"
                )

            # Ensure data exists before proceeding
            if client_ppu_data is not None and reference_ppu_data.size > 0:
                variables = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']

                # Percentile histograms (same as CMJ)
                for i, var in enumerate(variables):
                    formatted_var = var.replace('_', ' ')
                    doc.add_paragraph(f"{formatted_var} Comparison", style='Heading 2')
                    bar_image = generate_bar_graph(
                        var,
                        float(client_ppu_data[i]),
                        reference_ppu_data[:, i],
                        f'{formatted_var} Comparison',
                        tmpdirname
                    )
                    doc.add_picture(bar_image, width=Inches(6))

                # Force–Velocity scatter (same as CMJ)
                client_ppu_dict = {
                    'Force_at_PP': float(client_ppu_data[3]),
                    'Vel_at_PP'  : float(client_ppu_data[4]),
                }
                reference_ppu_df = pd.DataFrame(reference_ppu_data, columns=variables)
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                ppu_scatter = generate_scatter_plot(
                    client_ppu_dict, reference_ppu_df,
                    'Force_at_PP', 'Vel_at_PP',
                    'PPU: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(ppu_scatter, width=Inches(6))
            else:
                print("PPU: missing client or reference data; skipping figure set.")

        # ───────────────────────── SLV ───────────────────────        
        elif movement == 'SLV':
            # ── CLIENT ROWS (Left/Right) ───────────────────────────────────────
            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP
                FROM SLV WHERE name = ? AND side = 'Left'
            """, (client_name,))
            client_slvl_data = client_cursor.fetchone()

            client_cursor.execute("""
                SELECT JH_IN, PP_FORCEPLATE, PP_W_per_kg, Force_at_PP, Vel_at_PP
                FROM SLV WHERE name = ? AND side = 'Right'
            """, (client_name,))
            client_slvr_data = client_cursor.fetchone()

            # ── REFERENCE (keep side to filter/inspect if needed) ─────────────
            slv_vars_no_side = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
            ref_sql = f"SELECT side, {', '.join(slv_vars_no_side)} FROM SLV"
            reference_slv_df = pd.read_sql_query(ref_sql, reference_conn)
            print(f"SLV reference rows: {len(reference_slv_df)}")

            # ── POWER OVERLAYS (Left & Right) ─────────────────────────────────
            # We try common patterns. Adjust to your actual export names if needed.
            power_dir = r"D:\Athletic Screen 2.0\Output Files"

            # Left trials
            slv_left_files = (sorted(globmod.glob(os.path.join(power_dir, "*SLVL*_Power*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "SLV_Power_Left*.txt"))) or
                  sorted(globmod.glob(os.path.join(power_dir, "SLV_Left*_Power*.txt"))))

            # Right trials
            slv_right_files = (sorted(globmod.glob(os.path.join(power_dir, "*SLVR*_Power*.txt"))) or
                   sorted(globmod.glob(os.path.join(power_dir, "SLV_Power_Right*.txt"))) or
                   sorted(globmod.glob(os.path.join(power_dir, "SLV_Right*_Power*.txt"))))

            # --- SLV power files & analysis ---
            slv_files = find_power_files("SLV")
            if slv_files:
                slv_traces = [load_power_txt(pf) for pf in slv_files]
                add_power_analysis_section(
                    doc, movement="SLV", traces=slv_traces, fs_hz=1000,
                    tmpdirname=tmpdirname, reference_cursor=reference_cursor,
                    reference_table="SLV"
                )

            # ── HISTOGRAMS + SCATTER (with reference) ─────────────────────────
            if client_slvl_data and client_slvr_data and not reference_slv_df.empty:
                # Build name→value dicts so we don't rely on fragile positional indexes
                slv_cols = ['JH_IN', 'PP_FORCEPLATE', 'PP_W_per_kg', 'Force_at_PP', 'Vel_at_PP']
                client_slvl = dict(zip(slv_cols, map(float, client_slvl_data)))
                client_slvr = dict(zip(slv_cols, map(float, client_slvr_data)))
            
                # We want JH, PP, Force@PP (fixed), and Vel@PP
                for var in ['JH_IN', 'PP_FORCEPLATE', 'Force_at_PP', 'Vel_at_PP']:
                    doc.add_paragraph(f"{var.replace('_',' ')} Comparison (Left vs Right)",
                                      style='Heading 2')
            
                    # Reference distribution by column name
                    ref_col = reference_slv_df[var].to_numpy(dtype=float, copy=False)
            
                    # Client values by column name (correct variables now)
                    left_val  = client_slvl.get(var, np.nan)
                    right_val = client_slvr.get(var, np.nan)
            
                    hist = generate_slv_histogram(
                        var,
                        left_val,            # Left
                        right_val,           # Right
                        ref_col,
                        f'{var} Comparison',
                        tmpdirname
                    )
                    doc.add_picture(hist, width=Inches(6))
            
                # Scatter Force vs Velocity (merge both sides)
                client_slv_all = np.array([list(client_slvl.values()),
                                           list(client_slvr.values())], dtype=float)
                client_slv_dict = {
                    'Force_at_PP': [client_slvl['Force_at_PP'], client_slvr['Force_at_PP']],
                    'Vel_at_PP'  : [client_slvl['Vel_at_PP'],   client_slvr['Vel_at_PP']],
                }
                ref_for_scatter = reference_slv_df[['Force_at_PP', 'Vel_at_PP']]
                doc.add_paragraph("Force vs. Velocity Scatter Plot", style='Heading 2')
                slv_scatter = generate_scatter_plot(
                    client_slv_dict,
                    ref_for_scatter,
                    'Force_at_PP', 'Vel_at_PP',
                    'SLV: Force vs. Velocity', tmpdirname
                )
                doc.add_picture(slv_scatter, width=Inches(6))
            else:
                if not (client_slvl_data and client_slvr_data):
                    print("⚠️  Missing SLV client data – skipping SLV graphs.")
                if reference_slv_df.empty:
                    print("⚠️  SLV reference pull returned 0 rows.")
    
        # elif movement == 'NMT':
        #     # Fetch NMT data for the client (10s taps only)
        #     print("Entering NMT section…")
        #     client_cursor.execute("SELECT NUM_TAPS_10s FROM NMT WHERE name = ?", (client_name,))
        #     client_nmt_data = client_cursor.fetchone()
        # 
        #     reference_cursor.execute("SELECT NUM_TAPS_10s FROM NMT")
        #     reference_nmt_data = np.array(reference_cursor.fetchall(), dtype=float)
        # 
        #     print(f"NMT client row present: {client_nmt_data is not None}, reference rows: {len(reference_nmt_data)}")
        # 
        #     if client_nmt_data and reference_nmt_data.size > 0:
        #         nmt_var_label = 'NUM TAPS (10s)'
        #         doc.add_paragraph(f"{nmt_var_label} Comparison", style='Heading 2')
        # 
        #         # Pass table_hint="NMT" to avoid mis-inference
        #         nmt_image = generate_bar_graph(
        #             'NUM_TAPS_10s',
        #             float(client_nmt_data[0]),
        #             reference_nmt_data[:, 0],
        #             f'{nmt_var_label} Comparison',
        #             tmpdirname          # ← stop here
        #         )
        #         doc.add_picture(nmt_image, width=Inches(6))
        #     else:
        #         print("NMT: missing client or reference data; skipping figure.")


# Function to convert DOCX to images
def docx_to_images(docx_path, output_dir):
    # Extract text from the DOCX file
    text = docx2txt.process(docx_path)
    
    # Split the text into lines
    lines = text.splitlines()

    # Create a blank image with white background
    img_width, img_height = 1000, 1500
    image = Image.new('RGB', (img_width, img_height), color='white')
    draw = ImageDraw.Draw(image)

    # Use a simple font
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    # Draw the text onto the image
    padding = 20
    y_text = padding
    for line in lines:
        if y_text + padding > img_height:
            # Save the image and start a new one if the text exceeds the page height
            img_path = os.path.join(output_dir, f"page_{int(y_text / img_height)}.png")
            image.save(img_path)
            y_text = padding
            image = Image.new('RGB', (img_width, img_height), color='white')
            draw = ImageDraw.Draw(image)

        # Calculate text size and draw it
        text_bbox = draw.textbbox((0, 0), line, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        draw.text((padding, y_text), line, font=font, fill="black")
        y_text += text_height + padding

    # Save the last image
    img_path = os.path.join(output_dir, "final_page.png")
    image.save(img_path)

    return img_path

doc.save(output_filename)          # ← only one final save
print(f"Document saved at: {output_filename}")

# Close connections
client_conn.close()
reference_conn.close()

# Example usage
img_output_directory = r'G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Images'
os.makedirs(img_output_directory, exist_ok=True)

# Convert DOCX to images
img_path = docx_to_images(output_filename, img_output_directory)
print(f"Images saved at {img_path}")

Databases opened successfully.
Client Name: Jordan Driver
DJ reference rows: 143
SLV reference rows: 202
Document saved at: G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Athletic_Report_Jordan Driver_2025-11-07_All.docx
Images saved at G:\My Drive\Athletic Screen 2.0 Reports\College Reports\Images\final_page.png


In [4]:
import sqlite3
import time
import os

# --- Paths ---
source_db_path  = r"D:/Athletic Screen 2.0/Output Files/movement_database_v2.db"
output_folder   = r"D:/Athletic Screen 2.0/Output Files/"
target_databases = ["Athletic_Screen_College_data_v2.db"]
all_data_db_path = os.path.join(output_folder, "Athletic_Screen_All_data_v2.db")

# --- Retry helper (for "database is locked") ---
def retry_execute(func, retries=5, delay=1.0):
    while retries > 0:
        try:
            func()
            return
        except sqlite3.OperationalError as e:
            if "database is locked" in str(e).lower():
                print("Database is locked, retrying...")
                time.sleep(delay)
                retries -= 1
            else:
                raise
    raise Exception("Max retries reached. Database is still locked.")

# --- Open connections ---
source_conn = sqlite3.connect(source_db_path, timeout=10)
source_conn.execute("PRAGMA journal_mode=WAL;")
source_conn.execute("PRAGMA busy_timeout=5000;")
source_cursor = source_conn.cursor()

target_conns = {
    db_name: sqlite3.connect(os.path.join(output_folder, db_name), timeout=10)
    for db_name in target_databases
}
# also keep one combined "all" db
target_conns["all"] = sqlite3.connect(all_data_db_path, timeout=10)

# --- Make targets tolerant + fast ---
def prep_conn(conn):
    conn.execute("PRAGMA journal_mode=WAL;")
    conn.execute("PRAGMA busy_timeout=5000;")

for conn in target_conns.values():
    prep_conn(conn)

# --- Unique keys per table for idempotent UPSERTs ---
unique_keys = {
    "CMJ": ["name", "date", "trial_name"],
    "DJ" : ["name", "date", "trial_name"],
    "PPU": ["name", "date", "trial_name"],
    "SLV": ["name", "date", "trial_name", "side"],
    "NMT": ["name", "date", "trial_name"],
}

def table_exists(conn, table):
    try:
        conn.execute(f"SELECT 1 FROM {table} LIMIT 1")
        return True
    except sqlite3.OperationalError:
        return False

def get_columns(conn, table):
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table})")
    # returns list of (name, type)
    return [(r[1], r[2]) for r in cur.fetchall()]

def sync_table_schema_from_source(table_name):
    # read source columns/types
    src_cols = get_columns(source_conn, table_name)  # [(name, type), ...]
    if not src_cols:
        print(f"Source table {table_name} has no columns; skipping schema sync.")
        return

    for db_name, conn in target_conns.items():
        cur = conn.cursor()

        # Create table if missing (mirror source columns except autoincrement id)
        if not table_exists(conn, table_name):
            cols_sql = ", ".join(
                f"{col} {ctype or 'TEXT'}"
                for col, ctype in src_cols
                if col != "id"
            )
            cur.execute(f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    {cols_sql}
                )
            """)
            conn.commit()

        # Add any missing columns (this is how new power metrics get propagated)
        tgt_cols = dict(get_columns(conn, table_name))
        for col, ctype in src_cols:
            if col == "id":
                continue
            if col not in tgt_cols:
                cur.execute(f"ALTER TABLE {table_name} ADD COLUMN {col} {ctype or 'REAL'}")
        conn.commit()

        # Ensure unique index for upsert
        if table_name in unique_keys:
            # 1) dedupe existing rows so the unique index can be created
            dedupe_on_keys(conn, table_name, unique_keys[table_name])
        
            # 2) create the unique index (will succeed now)
            idx_cols = ",".join(unique_keys[table_name])
            cur.execute(
                f"CREATE UNIQUE INDEX IF NOT EXISTS idx_{table_name.lower()}_uniq "
                f"ON {table_name}({idx_cols})"
            )
            conn.commit()

            
def dedupe_on_keys(conn, table, key_cols):
    """
    Remove duplicate rows keeping the latest (highest rowid) per unique key.
    key_cols: list like ["name","date","trial_name"] (add "side" for SLV).
    """
    cols = ", ".join(key_cols)
    sql = f"""
        DELETE FROM {table}
        WHERE rowid NOT IN (
            SELECT MAX(rowid)
            FROM {table}
            GROUP BY {cols}
        )
    """
    conn.execute(sql)
    conn.commit()

def copy_table_data(table_name):
    # Pull from source
    try:
        source_cursor.execute(f"SELECT * FROM {table_name}")
    except sqlite3.OperationalError as e:
        print(f"Source table {table_name} not found: {e}. Skipping.")
        return

    rows = source_cursor.fetchall()
    if not rows:
        print(f"No rows to copy for {table_name}. Skipping.")
        return

    src_cols_all = [d[0] for d in source_cursor.description]  # includes 'id'

    # Make sure targets have ALL columns from source (metrics included)
    sync_table_schema_from_source(table_name)

    # For each target, compute intersection of columns and UPSERT
    for db_name, conn in target_conns.items():
        cur = conn.cursor()
        tgt_cols_all = [c for c, _ in get_columns(conn, table_name)]
        common_cols = [c for c in src_cols_all if c != "id" and c in tgt_cols_all]
        if not common_cols:
            print(f"{db_name}: No common columns for {table_name}. Skipping.")
            continue

        placeholders = ", ".join(["?"] * len(common_cols))
        col_list = ", ".join(common_cols)

        if table_name in unique_keys:
            uk = unique_keys[table_name]
            # Update all non-UK columns on conflict (so metrics refresh)
            set_cols = [c for c in common_cols if c not in uk]
            set_clause = ", ".join([f"{c}=excluded.{c}" for c in set_cols]) or ""
            on_conflict = f"ON CONFLICT({', '.join(uk)}) DO " + (f"UPDATE SET {set_clause}" if set_clause else "NOTHING")
        else:
            on_conflict = ""

        sql = f"INSERT INTO {table_name} ({col_list}) VALUES ({placeholders}) {on_conflict}"

        # align row values to the common column order
        idx_map = {c: src_cols_all.index(c) for c in common_cols}
        data = [tuple(row[idx_map[c]] for c in common_cols) for row in rows]

        retry_execute(lambda: cur.executemany(sql, data))
        conn.commit()
        print(f"Copied/Upserted {len(data)} rows to {table_name} in {db_name}")

# --- Tables to replicate ---
tables_to_copy = ["CMJ", "DJ", "PPU", "SLV", "NMT"]

# --- Run the copy ---
for table in tables_to_copy:
    copy_table_data(table)

# --- Close connections ---
source_conn.close()
for conn in target_conns.values():
    conn.close()

print("Data successfully copied to each target and combined database.")

# --- Optional: clear ASCII .txt exports after ingestion ---
ascii_folder = r"D:/Athletic Screen 2.0/Output Files/"
for filename in os.listdir(ascii_folder):
    if filename.lower().endswith(".txt"):
        file_path = os.path.join(ascii_folder, filename)
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")

print("All ASCII .txt files cleared after ingestion.")


Copied/Upserted 3 rows to CMJ in Athletic_Screen_College_data_v2.db
Copied/Upserted 3 rows to CMJ in all
Copied/Upserted 3 rows to DJ in Athletic_Screen_College_data_v2.db
Copied/Upserted 3 rows to DJ in all
Copied/Upserted 2 rows to PPU in Athletic_Screen_College_data_v2.db
Copied/Upserted 2 rows to PPU in all
Copied/Upserted 4 rows to SLV in Athletic_Screen_College_data_v2.db
Copied/Upserted 4 rows to SLV in all
No rows to copy for NMT. Skipping.
Data successfully copied to each target and combined database.
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ1.txt
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ1_Power.txt
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ2.txt
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ2_Power.txt
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ3.txt
Deleted: D:/Athletic Screen 2.0/Output Files/CMJ3_Power.txt
Deleted: D:/Athletic Screen 2.0/Output Files/DJ1.txt
Deleted: D:/Athletic Screen 2.0/Output Files/DJ1_Power.txt
Deleted: D:/Athletic Screen 2.

In [5]:
import shutil

# Paths
db_source_paths = [
    r"D:/Athletic Screen 2.0/Output Files/movement_database_v2.db",
    r"D:/Athletic Screen 2.0/Output Files/Athletic_Screen_All_data_v2.db",
    r"D:/Athletic Screen 2.0/Output Files/Athletic_Screen_College_data_v2.db"
]
destination_folder = r"G:/My Drive/Data/Athletic Screen Data"

# Copy each DB file to the destination
for db_path in db_source_paths:
    try:
        shutil.copy(db_path, destination_folder)
        print(f"Copied {db_path} → {destination_folder}")
    except Exception as e:
        print(f"Failed to copy {db_path}: {e}")


Copied D:/Athletic Screen 2.0/Output Files/movement_database_v2.db → G:/My Drive/Data/Athletic Screen Data
Copied D:/Athletic Screen 2.0/Output Files/Athletic_Screen_All_data_v2.db → G:/My Drive/Data/Athletic Screen Data
Copied D:/Athletic Screen 2.0/Output Files/Athletic_Screen_College_data_v2.db → G:/My Drive/Data/Athletic Screen Data
